In [ ]:
inputDir="ReadFiles"
fileName = "1_0.5_9.5_Iabc_consumator.txt"

In [ ]:
import path
import math
import cmath
import os
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pywt
import csv

In [ ]:
def readSingleFile(fileName, inputDir="ReadFiles"):
    result = ""
    if (fileName!=""):
        fileDir = os.path.join(os.path.join(os.getcwd(),inputDir),fileName)
        try:
            with open(fileDir,'r') as p:
                result=p.readlines()
        except:
            print('file not found')
            result = ""
    return result

def ReadSingleData(data=""):
    result = tuple()
    if (data!=""):
        time=[]
        input_A1=[]
        input_A2=[]
        input_A3=[]
        for line in data:
            time.append(float(line.replace('\n','').split('\t')[0]))
            input_A1.append(float(line.replace('\n','').split('\t')[1]))
            input_A2.append(float(line.replace('\n','').split('\t')[2]))
            input_A3.append(float(line.replace('\n','').split('\t')[3]))
        result = (time, input_A1, input_A2, input_A3)
    return result

def ExtractSingleData(data, N=1, frequency=50):
    time=data[0]
    input_A1=data[1]
    input_A2=data[2]
    input_A3=data[3]
    cycles = [math.floor(time[i]/(1/frequency)) for i in range(len(time))]
    result = list()
    for j in list(set(cycles))[0:-1]:
        count1=0
        input_A1_sin=0
        input_A1_cos=0
        input_A2_sin=0
        input_A2_cos=0
        input_A3_sin=0
        input_A3_cos=0
        for i in range(len(time)):
            if (j==cycles[i]):
                count1+=1
                input_A1_sin += input_A1[i]*(math.sin((2*math.pi/(1/frequency))*N*time[i]))
                input_A1_cos += input_A1[i]*(math.cos((2*math.pi/(1/frequency))*N*time[i]))
                input_A2_sin += input_A2[i]*(math.sin((2*math.pi/(1/frequency))*N*time[i]))
                input_A2_cos += input_A2[i]*(math.cos((2*math.pi/(1/frequency))*N*time[i]))
                input_A3_sin += input_A3[i]*(math.sin((2*math.pi/(1/frequency))*N*time[i]))
                input_A3_cos += input_A3[i]*(math.cos((2*math.pi/(1/frequency))*N*time[i]))
        result.append([j, math.sqrt((input_A1_sin * 2/count1)**2+ (input_A1_cos * 2/count1)**2)/math.sqrt(2), 
                              math.atan((input_A1_cos * 2/count1)/(input_A1_sin * 2/count1)),
                              math.sqrt((input_A2_sin * 2/count1)**2+ (input_A2_cos * 2/count1)**2)/math.sqrt(2),
                              math.atan((input_A2_cos * 2/count1)/(input_A2_sin * 2/count1)),
                              math.sqrt((input_A3_sin * 2/count1)**2+ (input_A3_cos * 2/count1)**2)/math.sqrt(2),
                              math.atan((input_A3_cos * 2/count1)/(input_A3_sin * 2/count1))])
    return result
def GetDataFromSingleFile(fileName, inputDir="ReadFiles", N=1, frequency=50):
    result = list()
    result.append(["fileName","cycle","Amplitudine-1","Phase-1","Amplitudine-2","Phase-2","Amplitudine-3","Phase-3"])
    aux=ExtractSingleData(ReadSingleData(readSingleFile(fileName,inputDir)),N,frequency)
    for item in aux:
        result.append((fileName, item[0], item[1], item[2], item[3], item[4], item[5], item[6]))
    return result

In [ ]:
def ReadMultipleFiles(inputDir="ReadFiles"):
    for folder,sub_folder,files in os.walk(os.path.join(os.getcwd(),inputDir)):
        files.sort()
        result=list()
        for nume_fisier in files:
            key1,value=nume_fisier.split('_',1)
            result.append([int(key1),nume_fisier])
        result.sort()
        result2=list()
        for item in result:
            fileDir = os.path.join(os.path.join(os.getcwd(),inputDir),item[1])
            result2.append([item[1],readSingleFile(item[1],inputDir)])  
    return result2

def ReadMultipleData(data):
    result = list()
    for item in data:
        result.append([item[0],ReadSingleData(item[1])])
    return result

def ExtractMultipleData(data, N=1, frequency=50):
    result = list()
    for item in data:
        result.append([item[0],ExtractSingleData(item[1],N,frequency)])
    return result  
def GetDataFromMultipleFile(inputDir="ReadFiles", N=1, frequency=50):
    result = list()
    aux=ExtractMultipleData(ReadMultipleData(ReadMultipleFiles(inputDir)),1,50)
    result.append(["fileName","cycle","Amplitudine-1","Phase-1","Amplitudine-2","Phase-2","Amplitudine-3","Phase-3"])
    for item in aux:
        for field in item[1]:
            result.append((item[0],field[0],field[1],field[2],field[3],field[4],field[5],field[6]))
    return result

In [ ]:
def ComputeContinueComponent(data, frequency=50):
    time=data[0]
    input_A1=data[1]
    input_A2=data[2]
    input_A3=data[3]
    cycles = [math.floor(time[i]/(1/frequency)) for i in range(len(time))]
    result = list()
    for j in list(set(cycles))[0:-1]:
        count1=0
        input_A1_value=0
        input_A2_value=0
        input_A3_value=0    
        for i in range(len(time)):
            if (j==cycles[i]):
                count1+=1
                input_A1_value += input_A1[i]
                input_A2_value += input_A2[i]
                input_A3_value += input_A3[i]
        result.append([j,input_A1_value * 1/count1, input_A2_value * 1/count1, input_A3_value * 1/count1])
    return result
def ComputeEfectiveValueComponent(data, frequency=50):
    time=data[0]
    input_A1=data[1]
    input_A2=data[2]
    input_A3=data[3]
    cycles = [math.floor(time[i]/(1/frequency)) for i in range(len(time))]
    result = list()
    for j in list(set(cycles))[0:-1]:
        count1=0
        input_A1_value=0
        input_A2_value=0
        input_A3_value=0    
        for i in range(len(time)):
            if (j==cycles[i]):
                count1+=1
                input_A1_value += input_A1[i]*input_A1[i]
                input_A2_value += input_A2[i]*input_A2[i]
                input_A3_value += input_A3[i]*input_A3[i]
        result.append([j,math.sqrt(input_A1_value * 1/count1), math.sqrt(input_A2_value * 1/count1), math.sqrt(input_A3_value * 1/count1)])
    return result
def ComputeDeformingRegime(effectiveComponent, continueComponent, firstArmonic):
    result=list()
    for itemId in range(len(firstArmonic)):
        aux1 = effectiveComponent[itemId][1]**2-continueComponent[itemId][1]**2 - firstArmonic[itemId][1]**2
        aux2 = effectiveComponent[itemId][2]**2-continueComponent[itemId][2]**2 - firstArmonic[itemId][3]**2
        aux3 = effectiveComponent[itemId][3]**2-continueComponent[itemId][3]**2 - firstArmonic[itemId][5]**2
        result.append([effectiveComponent[itemId][0], math.sqrt(abs(aux1)), math.sqrt(abs(aux2)), math.sqrt(abs(aux3))])
    return result
def ComputeDistorsionFactor(DeformingComponent,FirstArmonic):
    # THD
    result=list()
    for itemId in range(len(FirstArmonic)):
        aux1=DeformingComponent[itemId][1]*100/FirstArmonic[itemId][1]
        aux2=DeformingComponent[itemId][2]*100/FirstArmonic[itemId][3]
        aux3=DeformingComponent[itemId][3]*100/FirstArmonic[itemId][5]
        result.append([FirstArmonic[itemId][0],aux1,aux2,aux3])
    return result
def ComputeDistorsionContinueComponent(effectiveComponent, continueComponent):
    result=list()
    for itemId in range(len(effectiveComponent)):
        aux1=continueComponent[itemId][1]*100/effectiveComponent[itemId][1]
        aux2=continueComponent[itemId][2]*100/effectiveComponent[itemId][2]
        aux3=continueComponent[itemId][3]*100/effectiveComponent[itemId][3]
        result.append([effectiveComponent[itemId][0],aux1,aux2,aux3])
    return result
def ComputeDistorsionCurrentArmonic(effectiveComponent, FirstArmonic):
    result=list()
    for itemId in range(len(effectiveComponent)):
        aux1=FirstArmonic[itemId][1]*100/effectiveComponent[itemId][1]
        aux2=FirstArmonic[itemId][3]*100/effectiveComponent[itemId][2]
        aux3=FirstArmonic[itemId][5]*100/effectiveComponent[itemId][3]
        result.append([effectiveComponent[itemId][0],aux1,aux2,aux3])
    return result

def ComputeDeformingRegimeSingleValue(effectiveComponent, continueComponent, firstArmonic):
    return math.sqrt(abs(effectiveComponent**2-continueComponent**2 - firstArmonic**2))
def ComputeDistorsionFactorSingleValue(DeformingComponent,FirstArmonic):
    return DeformingComponent*100/FirstArmonic
def ComputeDistorsionContinueComponentSingleValue(effectiveComponent, continueComponent):
    return continueComponent*100/effectiveComponent
def ComputeDistorsionCurrentArmonicSingleValue(effectiveComponent, FirstArmonic):
    return FirstArmonic*100/effectiveComponent


In [ ]:
def ComponentPhaseDirect(phase, k):
    return phase + (2*math.pi / 3)*(k-1)
def ComponentPhaseInverse(phase, k):
    return phase + (2*math.pi / 3)*(4-k)

In [ ]:
def ComputeDirectComponent(y1,ph1,y2,ph2,y3,ph3):
    aux1 = y1*math.cos(ComponentPhaseDirect(ph1,1))+y2*math.cos(ComponentPhaseDirect(ph2,2))+y3*math.cos(ComponentPhaseDirect(ph3,3))
    aux2 = y1*math.sin(ComponentPhaseDirect(ph1,1))+y2*math.sin(ComponentPhaseDirect(ph2,2))+y3*math.sin(ComponentPhaseDirect(ph3,3))
    return (1/3)*(math.sqrt(aux1**2+aux2**2))
def ComputeInverseComponent(y1,ph1,y2,ph2,y3,ph3):
    aux1 = y1*math.cos(ComponentPhaseInverse(ph1,1))+y2*math.cos(ComponentPhaseInverse(ph2,2))+y3*math.cos(ComponentPhaseInverse(ph3,3))
    aux2 = y1*math.sin(ComponentPhaseInverse(ph1,1))+y2*math.sin(ComponentPhaseInverse(ph2,2))+y3*math.sin(ComponentPhaseInverse(ph3,3))
    return (1/3)*(math.sqrt(aux1**2+aux2**2))
def ComputeZeroComponent(y1,ph1,y2,ph2,y3,ph3):
    aux1 = y1*math.cos(ph1)+y2*math.cos(ph2)+y3*math.cos(ph3)
    aux2 = y1*math.sin(ph1)+y2*math.sin(ph2)+y3*math.sin(ph3)
    return (1/3)*(math.sqrt(aux1**2+aux2**2))
def CoefficeintNegativeAsimetry(inverse, direct):
    if direct != 0:
        return (inverse*100/direct)
    else:
        return 0
def CoefficeintZeroAsimetry(zero, direct):
    if direct != 0:
        return (zero*100/direct)
    else:
        return 0    

In [ ]:
def Create_Complex(Ampli,Phase,inp='rad'):
    if (inp=='rad'):
        return complex(Ampli*math.cos(Phase),Ampli*math.sin(Phase))
    else:
        return complex(Ampli*math.cos(Phase*math.pi/180),Ampli*math.sin(Phase*math.pi/180))

In [ ]:
def ComputeZeroComponentComplex(y1,ph1,y2,ph2,y3,ph3):
    aux1 = ( Create_Complex(y1,ph1) + Create_Complex(y2,ph2) + Create_Complex(y3,ph3) ) / 3
    return (math.sqrt(aux1.real**2+aux1.imag**2),np.angle(aux1) )
def ComputeDirectComponentComplex(y1,ph1,y2,ph2,y3,ph3):
    aux1 = ( Create_Complex(y1,ph1) + Create_Complex(y2,ph2) * Create_Complex(1,math.pi*2/3) + Create_Complex(y3,ph3) * Create_Complex(1,math.pi*4/3) ) / 3
    return (math.sqrt(aux1.real**2+aux1.imag**2),np.angle(aux1) )
def ComputeInverseComponentComplex(y1,ph1,y2,ph2,y3,ph3):
    aux1 = ( Create_Complex(y1,ph1) + Create_Complex(y2,ph2) * Create_Complex(1,math.pi*4/3) + Create_Complex(y3,ph3) * Create_Complex(1,math.pi*2/3) ) / 3
    return (math.sqrt(aux1.real**2+aux1.imag**2),np.angle(aux1) )

In [ ]:
def ComputeAsymetryCoefficientsSingleFile(fileName, inputDir="ReadFiles", N=1, frequency=50):
    result = list()
    result.append(["fileName", "cycle","DirectComponent","InverseComponent","ZeroComponent","CoefficeintNegativeAsimetry","CoefficeintZeroAsimetry"])
    aux=GetDataFromSingleFile(fileName, inputDir, N, frequency)
    for item in aux[1:]:
        a1 = ComputeDirectComponent(item[2],item[3],item[4],item[5],item[6],item[7])
        a2 = ComputeInverseComponent(item[2],item[3],item[4],item[5],item[6],item[7])
        a3 = ComputeZeroComponent(item[2],item[3],item[4],item[5],item[6],item[7])
        a4 = CoefficeintNegativeAsimetry(a2,a1)
        a5 = CoefficeintZeroAsimetry(a3,a1)
        result.append((item[0], item[1], a1, a2, a3, a4, a5))
    return result
def ComputeAsymetryCoefficientsSingleFileComplex(fileName, inputDir="ReadFiles", N=1, frequency=50):
    result = list()
    result.append(["fileName", "cycle","DirectComponent_Value","DirectComponent_Angle","InverseComponent_Value",
                   "InverseComponent_Angle","ZeroComponent_Value","ZeroComponent_Angle","CoefficeintNegativeAsimetry","CoefficeintZeroAsimetry"])
    aux=GetDataFromSingleFile(fileName, inputDir, N, frequency)
    for item in aux[1:]:
        a1,ph1 = ComputeDirectComponentComplex(item[2],item[3],item[4],item[5],item[6],item[7])
        a2,ph2 = ComputeInverseComponentComplex(item[2],item[3],item[4],item[5],item[6],item[7])
        a3,ph3 = ComputeZeroComponentComplex(item[2],item[3],item[4],item[5],item[6],item[7])
        a4 = CoefficeintNegativeAsimetry(a2,a1)
        a5 = CoefficeintZeroAsimetry(a3,a1)
        result.append((item[0], item[1], a1, ph1, a2, ph2, a3, ph3, a4, a5))
    return result

In [ ]:
inputDir="ReadFiles"
fileName = "1_0.5_9.5_Iabc_consumator.txt"

In [ ]:
def ComputeAsymetryCoefficientsMultipleFileComplex(inputDir="ReadFiles", N=1, frequency=50):
    result = list()
    result.append(["fileName", "cycle","DirectComponent","InverseComponent","ZeroComponent","CoefficeintNegativeAsimetry","CoefficeintZeroAsimetry"])
    aux=GetDataFromMultipleFile(inputDir, N, frequency)
    for item in aux[1:]:
        a1 = ComputeDirectComponentComplex(item[2],item[3],item[4],item[5],item[6],item[7])
        a2 = ComputeInverseComponentComplex(item[2],item[3],item[4],item[5],item[6],item[7])
        a3 = ComputeZeroComponentComplex(item[2],item[3],item[4],item[5],item[6],item[7])
        a4 = CoefficeintNegativeAsimetry(a2,a1)
        a5 = CoefficeintZeroAsimetry(a3,a1)
        result.append((item[0], item[1], a1, a2, a3, a4, a5))
    return result
def ComputeAsymetryCoefficientsMultipleFile(inputDir="ReadFiles", N=1, frequency=50):
    result = list()
    result.append(["fileName", "cycle","DirectComponent_Value","DirectComponent_Angle","InverseComponent_Value",
                   "InverseComponent_Angle","ZeroComponent_Value","ZeroComponent_Angle","CoefficeintNegativeAsimetry","CoefficeintZeroAsimetry"])
    aux=GetDataFromMultipleFile(inputDir, N, frequency)
    for item in aux[1:]:
        a1,ph1 = ComputeDirectComponentComplex(item[2],item[3],item[4],item[5],item[6],item[7])
        a2,ph2 = ComputeInverseComponentComplex(item[2],item[3],item[4],item[5],item[6],item[7])
        a3,ph3 = ComputeZeroComponentComplex(item[2],item[3],item[4],item[5],item[6],item[7])
        a4 = CoefficeintNegativeAsimetry(a2,a1)
        a5 = CoefficeintZeroAsimetry(a3,a1)
        result.append((item[0], item[1], a1, ph1, a2, ph2, a3, ph3, a4, a5))
    return result

In [ ]:
def ComputeHarmonicCoefficientSingleFile(fileName, inputDir="ReadFiles", N=1, frequency=50):
    input_data=ReadSingleData(readSingleFile(fileName,inputDir))
    firstArmonicData=ExtractSingleData(input_data,N,frequency)
    continueComponentData = ComputeContinueComponent(input_data,frequency)
    effectiveComponentData = ComputeEfectiveValueComponent(input_data,frequency)
    result=list()
    result.append(["fileName", "cycle","ContinueComponent_PhaseA","ContinueComponent_PhaseB","ContinueComponent_PhaseC",
                "EffectiveComponent_PhaseA","EffectiveComponent_PhaseB", "EffectiveComponent_PhaseC",
                "FirstArmonic_PhaseA","FirstArmonic_PhaseB","FirstArmonic_PhaseC",
                "DeformingComponent_PhaseA","DeformingComponent_PhaseB","DeformingComponent_PhaseC",
                "CoefficientDistorsion_PhaseA","CoefficientDistorsion_PhaseB","CoefficientDistorsion_PhaseC",
                "CoefficientDistorsionContinue_PhaseA","CoefficientDistorsionContinue_PhaseB","CoefficientDistorsionContinue_PhaseC",
                "CoefficientDistorsionFirstArmonic_PhaseA","CoefficientDistorsionFirstArmonic_PhaseB","CoefficientDistorsionFirstArmonic_PhaseC"])
    for i in range(len(firstArmonicData)):
        deforming1 = ComputeDeformingRegimeSingleValue(effectiveComponentData[i][1],continueComponentData[i][1],firstArmonicData[i][1])
        deforming2 = ComputeDeformingRegimeSingleValue(effectiveComponentData[i][2],continueComponentData[i][2],firstArmonicData[i][3])
        deforming3 = ComputeDeformingRegimeSingleValue(effectiveComponentData[i][3],continueComponentData[i][3],firstArmonicData[i][5])
        result.append((fileName,firstArmonicData[i][0],continueComponentData[i][1],continueComponentData[i][2],continueComponentData[i][3],
                      effectiveComponentData[i][1],effectiveComponentData[i][2],effectiveComponentData[i][3],
                      firstArmonicData[i][1],firstArmonicData[i][3],firstArmonicData[i][5],
                      deforming1, deforming2, deforming3,
                      ComputeDistorsionFactorSingleValue(deforming1,firstArmonicData[i][1]),
                       ComputeDistorsionFactorSingleValue(deforming2,firstArmonicData[i][3]),
                       ComputeDistorsionFactorSingleValue(deforming3,firstArmonicData[i][5]),
                       ComputeDistorsionContinueComponentSingleValue(effectiveComponentData[i][1],continueComponentData[i][1]),
                       ComputeDistorsionContinueComponentSingleValue(effectiveComponentData[i][2],continueComponentData[i][2]),
                       ComputeDistorsionContinueComponentSingleValue(effectiveComponentData[i][3],continueComponentData[i][3]),
                       ComputeDistorsionCurrentArmonicSingleValue(effectiveComponentData[i][1],firstArmonicData[i][1]),
                       ComputeDistorsionCurrentArmonicSingleValue(effectiveComponentData[i][2],firstArmonicData[i][3]),
                       ComputeDistorsionCurrentArmonicSingleValue(effectiveComponentData[i][3],firstArmonicData[i][5])
                      ))
    return result

In [ ]:
def ComputeHarmonicCoefficientMultipleFile(inputDir="ReadFiles", N=1, frequency=50):
    inputDataMultipleFile = ReadMultipleData(ReadMultipleFiles(inputDir))
    result=list()
    result.append(["fileName", "cycle","ContinueComponent_PhaseA","ContinueComponent_PhaseB","ContinueComponent_PhaseC",
                "EffectiveComponent_PhaseA","EffectiveComponent_PhaseB", "EffectiveComponent_PhaseC",
                "FirstArmonic_PhaseA","FirstArmonic_PhaseB","FirstArmonic_PhaseC",
                "DeformingComponent_PhaseA","DeformingComponent_PhaseB","DeformingComponent_PhaseC",
                "CoefficientDistorsion_PhaseA","CoefficientDistorsion_PhaseB","CoefficientDistorsion_PhaseC",
                "CoefficientDistorsionContinue_PhaseA","CoefficientDistorsionContinue_PhaseB","CoefficientDistorsionContinue_PhaseC",
                "CoefficientDistorsionFirstArmonic_PhaseA","CoefficientDistorsionFirstArmonic_PhaseB","CoefficientDistorsionFirstArmonic_PhaseC"])
    for idf in range(len(inputDataMultipleFile)):
        fileName=inputDataMultipleFile[idf][0]
        inputData=inputDataMultipleFile[idf][1]
        firstArmonicData=ExtractSingleData(inputData,N,frequency)
        continueComponentData = ComputeContinueComponent(inputData,frequency)
        effectiveComponentData = ComputeEfectiveValueComponent(inputData,frequency)
        for i in range(len(firstArmonicData)):
            deforming1 = ComputeDeformingRegimeSingleValue(effectiveComponentData[i][1],continueComponentData[i][1],firstArmonicData[i][1])
            deforming2 = ComputeDeformingRegimeSingleValue(effectiveComponentData[i][2],continueComponentData[i][2],firstArmonicData[i][3])
            deforming3 = ComputeDeformingRegimeSingleValue(effectiveComponentData[i][3],continueComponentData[i][3],firstArmonicData[i][5])
            result.append((fileName,firstArmonicData[i][0],continueComponentData[i][1],continueComponentData[i][2],continueComponentData[i][3],
                      effectiveComponentData[i][1],effectiveComponentData[i][2],effectiveComponentData[i][3],
                      firstArmonicData[i][1],firstArmonicData[i][3],firstArmonicData[i][5],
                      deforming1, deforming2, deforming3,
                      ComputeDistorsionFactorSingleValue(deforming1,firstArmonicData[i][1]),
                       ComputeDistorsionFactorSingleValue(deforming2,firstArmonicData[i][3]),
                       ComputeDistorsionFactorSingleValue(deforming3,firstArmonicData[i][5]),
                       ComputeDistorsionContinueComponentSingleValue(effectiveComponentData[i][1],continueComponentData[i][1]),
                       ComputeDistorsionContinueComponentSingleValue(effectiveComponentData[i][2],continueComponentData[i][2]),
                       ComputeDistorsionContinueComponentSingleValue(effectiveComponentData[i][3],continueComponentData[i][3]),
                       ComputeDistorsionCurrentArmonicSingleValue(effectiveComponentData[i][1],firstArmonicData[i][1]),
                       ComputeDistorsionCurrentArmonicSingleValue(effectiveComponentData[i][2],firstArmonicData[i][3]),
                       ComputeDistorsionCurrentArmonicSingleValue(effectiveComponentData[i][3],firstArmonicData[i][5])
                      ))
    return result

In [ ]:
def ComputeHarmonicAsymetrySingleFile(fileName, inputDir="ReadFiles", N=1, frequency=50):
    input_data=ReadSingleData(readSingleFile(fileName,inputDir))
    firstArmonicData=ExtractSingleData(input_data,N,frequency)
    continueComponentData = ComputeContinueComponent(input_data,frequency)
    effectiveComponentData = ComputeEfectiveValueComponent(input_data,frequency)
    result=list()
    result.append(["fileName", "cycle","ContinueComponent_PhaseA","ContinueComponent_PhaseB","ContinueComponent_PhaseC",
                "EffectiveComponent_PhaseA","EffectiveComponent_PhaseB", "EffectiveComponent_PhaseC",
                "FirstArmonic_PhaseA_Value","FirstArmonic_PhaseA_Angle",
                   "FirstArmonic_PhaseB_Value","FirstArmonic_PhaseB_Angle",
                   "FirstArmonic_PhaseC_Value","FirstArmonic_PhaseC_Angle",
                "DeformingComponent_PhaseA","DeformingComponent_PhaseB","DeformingComponent_PhaseC",
                "CoefficientDistorsion_PhaseA","CoefficientDistorsion_PhaseB","CoefficientDistorsion_PhaseC",
                "CoefficientDistorsionContinue_PhaseA","CoefficientDistorsionContinue_PhaseB","CoefficientDistorsionContinue_PhaseC",
                "CoefficientDistorsionFirstArmonic_PhaseA","CoefficientDistorsionFirstArmonic_PhaseB","CoefficientDistorsionFirstArmonic_PhaseC",
                "DirectComponent_Value","DirectComponent_Angle","InverseComponent_Value","InverseComponent_Angle",
                   "ZeroComponent_Value","ZeroComponent_Angle","CoefficeintNegativeAsimetry","CoefficeintZeroAsimetry"])
    for i in range(len(firstArmonicData)):
        deforming1 = ComputeDeformingRegimeSingleValue(effectiveComponentData[i][1],continueComponentData[i][1],firstArmonicData[i][1])
        deforming2 = ComputeDeformingRegimeSingleValue(effectiveComponentData[i][2],continueComponentData[i][2],firstArmonicData[i][3])
        deforming3 = ComputeDeformingRegimeSingleValue(effectiveComponentData[i][3],continueComponentData[i][3],firstArmonicData[i][5])
        a1,ph1 = ComputeDirectComponentComplex(firstArmonicData[i][1],firstArmonicData[i][2],firstArmonicData[i][3],firstArmonicData[i][4],firstArmonicData[i][5],firstArmonicData[i][6])
        a2,ph2 = ComputeInverseComponentComplex(firstArmonicData[i][1],firstArmonicData[i][2],firstArmonicData[i][3],firstArmonicData[i][4],firstArmonicData[i][5],firstArmonicData[i][6])
        a3,ph3 = ComputeZeroComponentComplex(firstArmonicData[i][1],firstArmonicData[i][2],firstArmonicData[i][3],firstArmonicData[i][4],firstArmonicData[i][5],firstArmonicData[i][6])
        result.append((fileName,firstArmonicData[i][0],continueComponentData[i][1],continueComponentData[i][2],continueComponentData[i][3],
                      effectiveComponentData[i][1],effectiveComponentData[i][2],effectiveComponentData[i][3],
                      firstArmonicData[i][1],firstArmonicData[i][2],firstArmonicData[i][3],firstArmonicData[i][4]
                       ,firstArmonicData[i][5],firstArmonicData[i][6],
                      deforming1, deforming2, deforming3,
                      ComputeDistorsionFactorSingleValue(deforming1,firstArmonicData[i][1]),
                       ComputeDistorsionFactorSingleValue(deforming2,firstArmonicData[i][3]),
                       ComputeDistorsionFactorSingleValue(deforming3,firstArmonicData[i][5]),
                       ComputeDistorsionContinueComponentSingleValue(effectiveComponentData[i][1],continueComponentData[i][1]),
                       ComputeDistorsionContinueComponentSingleValue(effectiveComponentData[i][2],continueComponentData[i][2]),
                       ComputeDistorsionContinueComponentSingleValue(effectiveComponentData[i][3],continueComponentData[i][3]),
                       ComputeDistorsionCurrentArmonicSingleValue(effectiveComponentData[i][1],firstArmonicData[i][1]),
                       ComputeDistorsionCurrentArmonicSingleValue(effectiveComponentData[i][2],firstArmonicData[i][3]),
                       ComputeDistorsionCurrentArmonicSingleValue(effectiveComponentData[i][3],firstArmonicData[i][5])
                       , a1, ph1, a2, ph2, a3, ph3, CoefficeintNegativeAsimetry(a2,a1), CoefficeintZeroAsimetry(a3,a1)
                      ))
    return result

def ComputeHarmonicCoefficientMultipleFile(inputDir="ReadFiles", N=1, frequency=50):
    inputDataMultipleFile = ReadMultipleData(ReadMultipleFiles(inputDir))
    result=list()
    result.append(["fileName", "cycle","ContinueComponent_PhaseA","ContinueComponent_PhaseB","ContinueComponent_PhaseC",
                "EffectiveComponent_PhaseA","EffectiveComponent_PhaseB", "EffectiveComponent_PhaseC",
                   "FirstArmonic_PhaseA_Value","FirstArmonic_PhaseA_Angle",
                   "FirstArmonic_PhaseB_Value","FirstArmonic_PhaseB_Angle",
                   "FirstArmonic_PhaseC_Value","FirstArmonic_PhaseC_Angle",
                "DeformingComponent_PhaseA","DeformingComponent_PhaseB","DeformingComponent_PhaseC",
                "CoefficientDistorsion_PhaseA","CoefficientDistorsion_PhaseB","CoefficientDistorsion_PhaseC",
                "CoefficientDistorsionContinue_PhaseA","CoefficientDistorsionContinue_PhaseB","CoefficientDistorsionContinue_PhaseC",
                "CoefficientDistorsionFirstArmonic_PhaseA","CoefficientDistorsionFirstArmonic_PhaseB","CoefficientDistorsionFirstArmonic_PhaseC",
                 "DirectComponent_Value","DirectComponent_Angle","InverseComponent_Value","InverseComponent_Angle",
            "ZeroComponent_Value","ZeroComponent_Angle","CoefficeintNegativeAsimetry","CoefficeintZeroAsimetry"])
    for idf in range(len(inputDataMultipleFile)):
        finleName=inputDataMultipleFile[idf][0]
        inputData=inputDataMultipleFile[idf][1]
        firstArmonicData=ExtractSingleData(inputData,N,frequency)
        continueComponentData = ComputeContinueComponent(inputData,frequency)
        effectiveComponentData = ComputeEfectiveValueComponent(inputData,frequency)
        for i in range(len(firstArmonicData)):
            deforming1 = ComputeDeformingRegimeSingleValue(effectiveComponentData[i][1],continueComponentData[i][1],firstArmonicData[i][1])
            deforming2 = ComputeDeformingRegimeSingleValue(effectiveComponentData[i][2],continueComponentData[i][2],firstArmonicData[i][3])
            deforming3 = ComputeDeformingRegimeSingleValue(effectiveComponentData[i][3],continueComponentData[i][3],firstArmonicData[i][5])
            a1,ph1 = ComputeDirectComponentComplex(firstArmonicData[i][1],firstArmonicData[i][2],firstArmonicData[i][3],firstArmonicData[i][4],firstArmonicData[i][5],firstArmonicData[i][6])
            a2,ph2 = ComputeInverseComponentComplex(firstArmonicData[i][1],firstArmonicData[i][2],firstArmonicData[i][3],firstArmonicData[i][4],firstArmonicData[i][5],firstArmonicData[i][6])
            a3,ph3 = ComputeZeroComponentComplex(firstArmonicData[i][1],firstArmonicData[i][2],firstArmonicData[i][3],firstArmonicData[i][4],firstArmonicData[i][5],firstArmonicData[i][6])
            result.append((finleName,firstArmonicData[i][0],continueComponentData[i][1],continueComponentData[i][2],continueComponentData[i][3],
                       effectiveComponentData[i][1],effectiveComponentData[i][2],effectiveComponentData[i][3],
                       firstArmonicData[i][1],firstArmonicData[i][2],firstArmonicData[i][3],
                       firstArmonicData[i][4],firstArmonicData[i][5],firstArmonicData[i][6],
                       deforming1, deforming2, deforming3,
                       ComputeDistorsionFactorSingleValue(deforming1,firstArmonicData[i][1]),
                       ComputeDistorsionFactorSingleValue(deforming2,firstArmonicData[i][3]),
                       ComputeDistorsionFactorSingleValue(deforming3,firstArmonicData[i][5]),
                       ComputeDistorsionContinueComponentSingleValue(effectiveComponentData[i][1],continueComponentData[i][1]),
                       ComputeDistorsionContinueComponentSingleValue(effectiveComponentData[i][2],continueComponentData[i][2]),
                       ComputeDistorsionContinueComponentSingleValue(effectiveComponentData[i][3],continueComponentData[i][3]),
                       ComputeDistorsionCurrentArmonicSingleValue(effectiveComponentData[i][1],firstArmonicData[i][1]),
                       ComputeDistorsionCurrentArmonicSingleValue(effectiveComponentData[i][2],firstArmonicData[i][3]),
                       ComputeDistorsionCurrentArmonicSingleValue(effectiveComponentData[i][3],firstArmonicData[i][5]), 
                           a1,ph1, a2,ph2, a3,ph3, CoefficeintNegativeAsimetry(a2,a1), CoefficeintZeroAsimetry(a3,a1)))
    return result

In [ ]:
def ExportDataFrameAllDataSingleFile(fileName, outputFile="", inputDir="ReadFiles", outputDir="OutputFile", N=1, frequency=50):
    if (outputFile==""):
        outputFile = fileName.replace('.txt','_ExportToCSV.csv')
    dataList = ComputeHarmonicAsymetrySingleFile(fileName, inputDir, N, frequency)
    data=pd.DataFrame.from_records(dataList[1:],columns=dataList[0])
    data.insert(1,"FaultLocation_FromGenerator",data["fileName"].map(lambda x: float(x.split('_')[1])))
    data.insert(2,"FaultLocation_ToConsumer",data["fileName"].map(lambda x: float(x.split('_')[2])))
    isExist = os.path.exists(outputDir)
    if not isExist:
        os.makedirs(outputDir)
    outputDir = os.path.join(os.path.join(os.getcwd(),outputDir),outputFile)
    
    data.to_csv(outputDir)
    return data

def ExportDataFrameAllDataMultipleFileFile(inputDir="ReadFiles", outputDir="OutputFile", N=1, frequency=50):
    outputFile = inputDir + '_ExportToCSV.csv'
    dataList = ComputeHarmonicCoefficientMultipleFile(inputDir, N, frequency)
    data=pd.DataFrame.from_records(dataList[1:],columns=dataList[0])
    data.insert(1,"FaultLocation_FromGenerator",data["fileName"].map(lambda x: float(x.split('_')[1])))
    data.insert(2,"FaultLocation_ToConsumer",data["fileName"].map(lambda x: float(x.split('_')[2])))
    isExist = os.path.exists(outputDir)
    if not isExist:
        os.makedirs(outputDir)    
    outputDir = os.path.join(os.path.join(os.getcwd(),outputDir),outputFile)
    data.to_csv(outputDir)
    return data

In [ ]:
def ExportAllFilesIntoOneCSV(inputDir="ReadFiles", outputDir="OutputFile", N=1, frequency=50):
    outputFile = inputDir + '_ExportToCSV.csv'
    for folder,sub_folder,files in os.walk(os.path.join(os.getcwd(),inputDir)):
        result_Vc=list()
        result_Ic=list()
        result_Ig=list()
        result_Vg=list()
        result_Id=list()
        result_Vd=list()
        for nume_fisier in files:
            key1,value=nume_fisier.split('_',1)
            if (('consumator'.upper() in nume_fisier.upper()) and '_Iabc'.upper() in nume_fisier.upper()):
                result_Ic.append([int(key1),nume_fisier])
            if (('consumator'.upper() in nume_fisier.upper()) and '_Vabc'.upper() in nume_fisier.upper()):
                result_Vc.append([int(key1),nume_fisier])
            if (('generator'.upper() in nume_fisier.upper()) and '_Iabc'.upper() in nume_fisier.upper()):
                result_Ig.append([int(key1),nume_fisier])
            if (('generator'.upper() in nume_fisier.upper()) and '_Vabc'.upper() in nume_fisier.upper()):
                result_Vg.append([int(key1),nume_fisier])
            if (('defect'.upper() in nume_fisier.upper()) and '_Iabc'.upper() in nume_fisier.upper()):
                result_Id.append([int(key1),nume_fisier])
            if (('defect'.upper() in nume_fisier.upper()) and '_Vabc'.upper() in nume_fisier.upper()):
                result_Vd.append([int(key1),nume_fisier])
    result_Ic.sort()
    result_Vc.sort()
    result_Ig.sort()
    result_Vg.sort()
    result_Id.sort()
    result_Vd.sort()
    output_Ic=list()
    output_Vc=list()
    output_Ig=list()
    output_Vg=list()
    output_Id=list()
    output_Vd=list()
    output_Ic.append(("Iabc_consumator_fileName", "cycle","Iabc_consumator_ContinueComponent_PhaseA","Iabc_consumator_ContinueComponent_PhaseB","Iabc_consumator_ContinueComponent_PhaseC",
                    "Iabc_consumator_EffectiveComponent_PhaseA","Iabc_consumator_EffectiveComponent_PhaseB", "Iabc_consumator_EffectiveComponent_PhaseC",
                    "Iabc_consumator_FirstArmonic_PhaseA_Value","Iabc_consumator_FirstArmonic_PhaseA_Angle",
                          "Iabc_consumator_FirstArmonic_PhaseB_Value","Iabc_consumator_FirstArmonic_PhaseB_Angle",
                          "Iabc_consumator_FirstArmonic_PhaseC_Value","Iabc_consumator_FirstArmonic_PhaseC_Angle",
                    "Iabc_consumator_DeformingComponent_PhaseA","Iabc_consumator_DeformingComponent_PhaseB","Iabc_consumator_DeformingComponent_PhaseC",
                    "Iabc_consumator_CoefficientDistorsion_PhaseA","Iabc_consumator_CoefficientDistorsion_PhaseB","Iabc_consumator_CoefficientDistorsion_PhaseC",
                    "Iabc_consumator_CoefficientDistorsionContinue_PhaseA","Iabc_consumator_CoefficientDistorsionContinue_PhaseB","Iabc_consumator_CoefficientDistorsionContinue_PhaseC",
                    "Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA","Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB","Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Iabc_consumator_DirectComponent_Value","Iabc_consumator_DirectComponent_Angle",
                      "Iabc_consumator_InverseComponent_Value","Iabc_consumator_InverseComponent_Angle",
                      "Iabc_consumator_ZeroComponent_Value", "Iabc_consumator_ZeroComponent_Angle",
                      "Iabc_consumator_CoefficeintNegativeAsimetry","Iabc_consumator_CoefficeintZeroAsimetry"))
    output_Vc.append(("Vabc_consumator_fileName", "cycle","Vabc_consumator_ContinueComponent_PhaseA","Vabc_consumator_ContinueComponent_PhaseB","Vabc_consumator_ContinueComponent_PhaseC",
                    "Vabc_consumator_EffectiveComponent_PhaseA","Vabc_consumator_EffectiveComponent_PhaseB", "Vabc_consumator_EffectiveComponent_PhaseC",
                    "Vabc_consumator_FirstArmonic_PhaseA_Value","Vabc_consumator_FirstArmonic_PhaseA_Angle",
                          "Vabc_consumator_FirstArmonic_PhaseB_Value","Vabc_consumator_FirstArmonic_PhaseB_Angle",
                          "Vabc_consumator_FirstArmonic_PhaseC_Value","Vabc_consumator_FirstArmonic_PhaseC_Angle",
                    "Vabc_consumator_DeformingComponent_PhaseA","Vabc_consumator_DeformingComponent_PhaseB","Vabc_consumator_DeformingComponent_PhaseC",
                    "Vabc_consumator_CoefficientDistorsion_PhaseA","Vabc_consumator_CoefficientDistorsion_PhaseB","Vabc_consumator_CoefficientDistorsion_PhaseC",
                    "Vabc_consumator_CoefficientDistorsionContinue_PhaseA","Vabc_consumator_CoefficientDistorsionContinue_PhaseB","Vabc_consumator_CoefficientDistorsionContinue_PhaseC",
                    "Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA","Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB","Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Vabc_consumator_DirectComponent_Value","Vabc_consumator_DirectComponent_Angle",
                      "Vabc_consumator_InverseComponent_Value","Vabc_consumator_InverseComponent_Angle",
                      "Vabc_consumator_ZeroComponent_Value", "Vabc_consumator_ZeroComponent_Angle",
                      "Vabc_consumator_CoefficeintNegativeAsimetry","Vabc_consumator_CoefficeintZeroAsimetry"))
    output_Ig.append(("Iabc_generator_fileName", "cycle","Iabc_generator_ContinueComponent_PhaseA","Iabc_generator_ContinueComponent_PhaseB","Iabc_generator_ContinueComponent_PhaseC",
                    "Iabc_generator_EffectiveComponent_PhaseA","Iabc_generator_EffectiveComponent_PhaseB", "Iabc_generator_EffectiveComponent_PhaseC",
                    "Iabc_generator_FirstArmonic_PhaseA_Value","Iabc_generator_FirstArmonic_PhaseA_Angle",
                          "Iabc_generator_FirstArmonic_PhaseB_Value","Iabc_generator_FirstArmonic_PhaseB_Angle",
                          "Iabc_generator_FirstArmonic_PhaseC_Value","Iabc_generator_FirstArmonic_PhaseC_Angle",
                    "Iabc_generator_DeformingComponent_PhaseA","Iabc_generator_DeformingComponent_PhaseB","Iabc_generator_DeformingComponent_PhaseC",
                    "Iabc_generator_CoefficientDistorsion_PhaseA","Iabc_generator_CoefficientDistorsion_PhaseB","Iabc_generator_CoefficientDistorsion_PhaseC",
                    "Iabc_generator_CoefficientDistorsionContinue_PhaseA","Iabc_generator_CoefficientDistorsionContinue_PhaseB","Iabc_generator_CoefficientDistorsionContinue_PhaseC",
                    "Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseA","Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseB","Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Iabc_generator_DirectComponent_Value","Iabc_generator_DirectComponent_Angle",
                      "Iabc_generator_InverseComponent_Value","Iabc_generator_InverseComponent_Angle",
                      "Iabc_generator_ZeroComponent_Value", "Iabc_generator_ZeroComponent_Angle",
                      "Iabc_generator_CoefficeintNegativeAsimetry","Iabc_generator_CoefficeintZeroAsimetry"))
    output_Vg.append(("Vabc_generator_fileName", "cycle","Vabc_generator_ContinueComponent_PhaseA","Vabc_generator_ContinueComponent_PhaseB","Vabc_generator_ContinueComponent_PhaseC",
                    "Vabc_generator_EffectiveComponent_PhaseA","Vabc_generator_EffectiveComponent_PhaseB", "Vabc_generator_EffectiveComponent_PhaseC",
                    "Vabc_generator_FirstArmonic_PhaseA_Value","Vabc_generator_FirstArmonic_PhaseA_Angle",
                          "Vabc_generator_FirstArmonic_PhaseB_Value","Vabc_generator_FirstArmonic_PhaseB_Angle",
                          "Vabc_generator_FirstArmonic_PhaseC_Value","Vabc_generator_FirstArmonic_PhaseC_Angle",
                    "Vabc_generator_DeformingComponent_PhaseA","Vabc_generator_DeformingComponent_PhaseB","Vabc_generator_DeformingComponent_PhaseC",
                    "Vabc_generator_CoefficientDistorsion_PhaseA","Vabc_generator_CoefficientDistorsion_PhaseB","Vabc_generator_CoefficientDistorsion_PhaseC",
                    "Vabc_generator_CoefficientDistorsionContinue_PhaseA","Vabc_generator_CoefficientDistorsionContinue_PhaseB","Vabc_generator_CoefficientDistorsionContinue_PhaseC",
                    "Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseA","Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseB","Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Vabc_generator_DirectComponent_Value","Vabc_generator_DirectComponent_Angle",
                      "Vabc_generator_InverseComponent_Value","Vabc_generator_InverseComponent_Angle",
                      "Vabc_generator_ZeroComponent_Value", "Vabc_generator_ZeroComponent_Angle",
                      "Vabc_generator_CoefficeintNegativeAsimetry","Vabc_generator_CoefficeintZeroAsimetry"))
    output_Id.append(("Iabc_loc_defect_fileName", "cycle","Iabc_loc_defect_ContinueComponent_PhaseA","Iabc_loc_defect_ContinueComponent_PhaseB","Iabc_loc_defect_ContinueComponent_PhaseC",
                    "Iabc_loc_defect_EffectiveComponent_PhaseA","Iabc_loc_defect_EffectiveComponent_PhaseB", "Iabc_loc_defect_EffectiveComponent_PhaseC",
                    "Iabc_loc_defect_FirstArmonic_PhaseA_Value","Iabc_loc_defect_FirstArmonic_PhaseA_Angle",
                          "Iabc_loc_defect_FirstArmonic_PhaseB_Value","Iabc_loc_defect_FirstArmonic_PhaseB_Angle",
                          "Iabc_loc_defect_FirstArmonic_PhaseC_Value","Iabc_loc_defect_FirstArmonic_PhaseC_Angle",
                    "Iabc_loc_defect_DeformingComponent_PhaseA","Iabc_loc_defect_DeformingComponent_PhaseB","Iabc_loc_defect_DeformingComponent_PhaseC",
                    "Iabc_loc_defect_CoefficientDistorsion_PhaseA","Iabc_loc_defect_CoefficientDistorsion_PhaseB","Iabc_loc_defect_CoefficientDistorsion_PhaseC",
                    "Iabc_loc_defect_CoefficientDistorsionContinue_PhaseA","Iabc_loc_defect_CoefficientDistorsionContinue_PhaseB","Iabc_loc_defect_CoefficientDistorsionContinue_PhaseC",
                    "Iabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseA","Iabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseB","Iabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Iabc_loc_defect_DirectComponent_Value","Iabc_loc_defect_DirectComponent_Angle",
                      "Iabc_loc_defect_InverseComponent_Value","Iabc_loc_defect_InverseComponent_Angle",
                      "Iabc_loc_defect_ZeroComponent_Value", "Iabc_loc_defect_ZeroComponent_Angle",
                      "Iabc_loc_defect_CoefficeintNegativeAsimetry","Iabc_loc_defect_CoefficeintZeroAsimetry"))
    output_Vd.append(("Vabc_loc_defect_fileName", "cycle","Vabc_loc_defect_ContinueComponent_PhaseA","Vabc_loc_defect_ContinueComponent_PhaseB","Vabc_loc_defect_ContinueComponent_PhaseC",
                    "Vabc_loc_defect_EffectiveComponent_PhaseA","Vabc_loc_defect_EffectiveComponent_PhaseB", "Vabc_loc_defect_EffectiveComponent_PhaseC",
                    "Vabc_loc_defect_FirstArmonic_PhaseA_Value","Vabc_loc_defect_FirstArmonic_PhaseA_Angle",
                          "Vabc_loc_defect_FirstArmonic_PhaseB_Value","Vabc_loc_defect_FirstArmonic_PhaseB_Angle",
                          "Vabc_loc_defect_FirstArmonic_PhaseC_Value","Vabc_loc_defect_FirstArmonic_PhaseC_Angle",
                    "Vabc_loc_defect_DeformingComponent_PhaseA","Vabc_loc_defect_DeformingComponent_PhaseB","Vabc_loc_defect_DeformingComponent_PhaseC",
                    "Vabc_loc_defect_CoefficientDistorsion_PhaseA","Vabc_loc_defect_CoefficientDistorsion_PhaseB","Vabc_loc_defect_CoefficientDistorsion_PhaseC",
                    "Vabc_loc_defect_CoefficientDistorsionContinue_PhaseA","Vabc_loc_defect_CoefficientDistorsionContinue_PhaseB","Vabc_loc_defect_CoefficientDistorsionContinue_PhaseC",
                    "Vabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseA","Vabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseB","Vabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Vabc_loc_defect_DirectComponent_Value","Vabc_loc_defect_DirectComponent_Angle",
                      "Vabc_loc_defect_InverseComponent_Value","Vabc_loc_defect_InverseComponent_Angle",
                      "Vabc_loc_defect_ZeroComponent_Value", "Vabc_loc_defect_ZeroComponent_Angle",
                      "Vabc_loc_defect_CoefficeintNegativeAsimetry","Vabc_loc_defect_CoefficeintZeroAsimetry"))  
    for i in range(len(result_Ic)):
        output_Ic += ComputeHarmonicAsymetrySingleFile(result_Ic[i][1],inputDir,N,frequency)[1:]
        output_Vc += ComputeHarmonicAsymetrySingleFile(result_Vc[i][1],inputDir,N,frequency)[1:]
        output_Ig += ComputeHarmonicAsymetrySingleFile(result_Ig[i][1],inputDir,N,frequency)[1:]
        output_Vg += ComputeHarmonicAsymetrySingleFile(result_Vg[i][1],inputDir,N,frequency)[1:]
        output_Id += ComputeHarmonicAsymetrySingleFile(result_Id[i][1],inputDir,N,frequency)[1:]
        output_Vd += ComputeHarmonicAsymetrySingleFile(result_Vd[i][1],inputDir,N,frequency)[1:]
    final_result = list()
    for i in range(len(output_Ic)):
        final_result.append(output_Ig[i] + output_Vg[i] + output_Id[i] + output_Vd[i] + output_Ic[i]+output_Vc[i])
    data = pd.DataFrame.from_records(final_result[1:], columns=final_result[0])
    data.insert(1, "FaultLocation_FromGenerator", data["Iabc_consumator_fileName"].map(lambda x: float(x.split('_')[1])))
    data.insert(2, "FaultLocation_ToConsumer", data["Iabc_consumator_fileName"].map(lambda x: float(x.split('_')[2])))
    isExist = os.path.exists(outputDir)
    if not isExist:
        os.makedirs(outputDir)
    outputDir = os.path.join(os.path.join(os.getcwd(), outputDir), outputFile)

    data.to_csv(outputDir)
    return data

In [ ]:
def ExportAllFilesIntoOneCSV_V3(inputDir="ReadFiles", outputDir="OutputFile", N=1, frequency=50):
    outputFile = inputDir + '_ExportToCSV.csv'
    for folder,sub_folder,files in os.walk(os.path.join(os.getcwd(),inputDir)):
        result_Vc=list()
        result_Ic=list()
        result_Ig=list()
        result_Vg=list()
        result_Id=list()
        result_Vd=list()
        for nume_fisier in files:
            key1,value=nume_fisier.split('_',1)
            if (('consumator'.upper() in nume_fisier.upper()) and '_Iabc'.upper() in nume_fisier.upper()):
                result_Ic.append([int(key1),nume_fisier])
            if (('consumator'.upper() in nume_fisier.upper()) and '_Vabc'.upper() in nume_fisier.upper()):
                result_Vc.append([int(key1),nume_fisier])
            if (('generator'.upper() in nume_fisier.upper()) and '_Iabc'.upper() in nume_fisier.upper()):
                result_Ig.append([int(key1),nume_fisier])
            if (('generator'.upper() in nume_fisier.upper()) and '_Vabc'.upper() in nume_fisier.upper()):
                result_Vg.append([int(key1),nume_fisier])
            if (('defect'.upper() in nume_fisier.upper()) and '_Iabc'.upper() in nume_fisier.upper()):
                result_Id.append([int(key1),nume_fisier])
            if (('defect'.upper() in nume_fisier.upper()) and '_Vabc'.upper() in nume_fisier.upper()):
                result_Vd.append([int(key1),nume_fisier])
    result_Ic.sort()
    result_Vc.sort()
    result_Ig.sort()
    result_Vg.sort()
    result_Id.sort()
    result_Vd.sort()
    output_Ic=list()
    output_Vc=list()
    output_Ig=list()
    output_Vg=list()
    output_Id=list()
    output_Vd=list()
    output_Ic.append(("Iabc_consumator_fileName", "cycle","Iabc_consumator_ContinueComponent_PhaseA","Iabc_consumator_ContinueComponent_PhaseB","Iabc_consumator_ContinueComponent_PhaseC",
                    "Iabc_consumator_EffectiveComponent_PhaseA","Iabc_consumator_EffectiveComponent_PhaseB", "Iabc_consumator_EffectiveComponent_PhaseC",
                    "Iabc_consumator_FirstArmonic_PhaseA_Value","Iabc_consumator_FirstArmonic_PhaseA_Angle",
                          "Iabc_consumator_FirstArmonic_PhaseB_Value","Iabc_consumator_FirstArmonic_PhaseB_Angle",
                          "Iabc_consumator_FirstArmonic_PhaseC_Value","Iabc_consumator_FirstArmonic_PhaseC_Angle",
                    "Iabc_consumator_DeformingComponent_PhaseA","Iabc_consumator_DeformingComponent_PhaseB","Iabc_consumator_DeformingComponent_PhaseC",
                    "Iabc_consumator_CoefficientDistorsion_PhaseA","Iabc_consumator_CoefficientDistorsion_PhaseB","Iabc_consumator_CoefficientDistorsion_PhaseC",
                    "Iabc_consumator_CoefficientDistorsionContinue_PhaseA","Iabc_consumator_CoefficientDistorsionContinue_PhaseB","Iabc_consumator_CoefficientDistorsionContinue_PhaseC",
                    "Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA","Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB","Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Iabc_consumator_DirectComponent_Value","Iabc_consumator_DirectComponent_Angle",
                      "Iabc_consumator_InverseComponent_Value","Iabc_consumator_InverseComponent_Angle",
                      "Iabc_consumator_ZeroComponent_Value", "Iabc_consumator_ZeroComponent_Angle",
                      "Iabc_consumator_CoefficeintNegativeAsimetry","Iabc_consumator_CoefficeintZeroAsimetry"))
    output_Vc.append(("Vabc_consumator_fileName", "cycle","Vabc_consumator_ContinueComponent_PhaseA","Vabc_consumator_ContinueComponent_PhaseB","Vabc_consumator_ContinueComponent_PhaseC",
                    "Vabc_consumator_EffectiveComponent_PhaseA","Vabc_consumator_EffectiveComponent_PhaseB", "Vabc_consumator_EffectiveComponent_PhaseC",
                    "Vabc_consumator_FirstArmonic_PhaseA_Value","Vabc_consumator_FirstArmonic_PhaseA_Angle",
                          "Vabc_consumator_FirstArmonic_PhaseB_Value","Vabc_consumator_FirstArmonic_PhaseB_Angle",
                          "Vabc_consumator_FirstArmonic_PhaseC_Value","Vabc_consumator_FirstArmonic_PhaseC_Angle",
                    "Vabc_consumator_DeformingComponent_PhaseA","Vabc_consumator_DeformingComponent_PhaseB","Vabc_consumator_DeformingComponent_PhaseC",
                    "Vabc_consumator_CoefficientDistorsion_PhaseA","Vabc_consumator_CoefficientDistorsion_PhaseB","Vabc_consumator_CoefficientDistorsion_PhaseC",
                    "Vabc_consumator_CoefficientDistorsionContinue_PhaseA","Vabc_consumator_CoefficientDistorsionContinue_PhaseB","Vabc_consumator_CoefficientDistorsionContinue_PhaseC",
                    "Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA","Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB","Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Vabc_consumator_DirectComponent_Value","Vabc_consumator_DirectComponent_Angle",
                      "Vabc_consumator_InverseComponent_Value","Vabc_consumator_InverseComponent_Angle",
                      "Vabc_consumator_ZeroComponent_Value", "Vabc_consumator_ZeroComponent_Angle",
                      "Vabc_consumator_CoefficeintNegativeAsimetry","Vabc_consumator_CoefficeintZeroAsimetry"))
    output_Ig.append(("Iabc_generator_fileName", "cycle","Iabc_generator_ContinueComponent_PhaseA","Iabc_generator_ContinueComponent_PhaseB","Iabc_generator_ContinueComponent_PhaseC",
                    "Iabc_generator_EffectiveComponent_PhaseA","Iabc_generator_EffectiveComponent_PhaseB", "Iabc_generator_EffectiveComponent_PhaseC",
                    "Iabc_generator_FirstArmonic_PhaseA_Value","Iabc_generator_FirstArmonic_PhaseA_Angle",
                          "Iabc_generator_FirstArmonic_PhaseB_Value","Iabc_generator_FirstArmonic_PhaseB_Angle",
                          "Iabc_generator_FirstArmonic_PhaseC_Value","Iabc_generator_FirstArmonic_PhaseC_Angle",
                    "Iabc_generator_DeformingComponent_PhaseA","Iabc_generator_DeformingComponent_PhaseB","Iabc_generator_DeformingComponent_PhaseC",
                    "Iabc_generator_CoefficientDistorsion_PhaseA","Iabc_generator_CoefficientDistorsion_PhaseB","Iabc_generator_CoefficientDistorsion_PhaseC",
                    "Iabc_generator_CoefficientDistorsionContinue_PhaseA","Iabc_generator_CoefficientDistorsionContinue_PhaseB","Iabc_generator_CoefficientDistorsionContinue_PhaseC",
                    "Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseA","Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseB","Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Iabc_generator_DirectComponent_Value","Iabc_generator_DirectComponent_Angle",
                      "Iabc_generator_InverseComponent_Value","Iabc_generator_InverseComponent_Angle",
                      "Iabc_generator_ZeroComponent_Value", "Iabc_generator_ZeroComponent_Angle",
                      "Iabc_generator_CoefficeintNegativeAsimetry","Iabc_generator_CoefficeintZeroAsimetry"))
    output_Vg.append(("Vabc_generator_fileName", "cycle","Vabc_generator_ContinueComponent_PhaseA","Vabc_generator_ContinueComponent_PhaseB","Vabc_generator_ContinueComponent_PhaseC",
                    "Vabc_generator_EffectiveComponent_PhaseA","Vabc_generator_EffectiveComponent_PhaseB", "Vabc_generator_EffectiveComponent_PhaseC",
                    "Vabc_generator_FirstArmonic_PhaseA_Value","Vabc_generator_FirstArmonic_PhaseA_Angle",
                          "Vabc_generator_FirstArmonic_PhaseB_Value","Vabc_generator_FirstArmonic_PhaseB_Angle",
                          "Vabc_generator_FirstArmonic_PhaseC_Value","Vabc_generator_FirstArmonic_PhaseC_Angle",
                    "Vabc_generator_DeformingComponent_PhaseA","Vabc_generator_DeformingComponent_PhaseB","Vabc_generator_DeformingComponent_PhaseC",
                    "Vabc_generator_CoefficientDistorsion_PhaseA","Vabc_generator_CoefficientDistorsion_PhaseB","Vabc_generator_CoefficientDistorsion_PhaseC",
                    "Vabc_generator_CoefficientDistorsionContinue_PhaseA","Vabc_generator_CoefficientDistorsionContinue_PhaseB","Vabc_generator_CoefficientDistorsionContinue_PhaseC",
                    "Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseA","Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseB","Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Vabc_generator_DirectComponent_Value","Vabc_generator_DirectComponent_Angle",
                      "Vabc_generator_InverseComponent_Value","Vabc_generator_InverseComponent_Angle",
                      "Vabc_generator_ZeroComponent_Value", "Vabc_generator_ZeroComponent_Angle",
                      "Vabc_generator_CoefficeintNegativeAsimetry","Vabc_generator_CoefficeintZeroAsimetry"))
    output_Id.append(("Iabc_loc_defect_fileName", "cycle","Iabc_loc_defect_ContinueComponent_PhaseA","Iabc_loc_defect_ContinueComponent_PhaseB","Iabc_loc_defect_ContinueComponent_PhaseC",
                    "Iabc_loc_defect_EffectiveComponent_PhaseA","Iabc_loc_defect_EffectiveComponent_PhaseB", "Iabc_loc_defect_EffectiveComponent_PhaseC",
                    "Iabc_loc_defect_FirstArmonic_PhaseA_Value","Iabc_loc_defect_FirstArmonic_PhaseA_Angle",
                          "Iabc_loc_defect_FirstArmonic_PhaseB_Value","Iabc_loc_defect_FirstArmonic_PhaseB_Angle",
                          "Iabc_loc_defect_FirstArmonic_PhaseC_Value","Iabc_loc_defect_FirstArmonic_PhaseC_Angle",
                    "Iabc_loc_defect_DeformingComponent_PhaseA","Iabc_loc_defect_DeformingComponent_PhaseB","Iabc_loc_defect_DeformingComponent_PhaseC",
                    "Iabc_loc_defect_CoefficientDistorsion_PhaseA","Iabc_loc_defect_CoefficientDistorsion_PhaseB","Iabc_loc_defect_CoefficientDistorsion_PhaseC",
                    "Iabc_loc_defect_CoefficientDistorsionContinue_PhaseA","Iabc_loc_defect_CoefficientDistorsionContinue_PhaseB","Iabc_loc_defect_CoefficientDistorsionContinue_PhaseC",
                    "Iabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseA","Iabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseB","Iabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Iabc_loc_defect_DirectComponent_Value","Iabc_loc_defect_DirectComponent_Angle",
                      "Iabc_loc_defect_InverseComponent_Value","Iabc_loc_defect_InverseComponent_Angle",
                      "Iabc_loc_defect_ZeroComponent_Value", "Iabc_loc_defect_ZeroComponent_Angle",
                      "Iabc_loc_defect_CoefficeintNegativeAsimetry","Iabc_loc_defect_CoefficeintZeroAsimetry"))
    output_Vd.append(("Vabc_loc_defect_fileName", "cycle","Vabc_loc_defect_ContinueComponent_PhaseA","Vabc_loc_defect_ContinueComponent_PhaseB","Vabc_loc_defect_ContinueComponent_PhaseC",
                    "Vabc_loc_defect_EffectiveComponent_PhaseA","Vabc_loc_defect_EffectiveComponent_PhaseB", "Vabc_loc_defect_EffectiveComponent_PhaseC",
                    "Vabc_loc_defect_FirstArmonic_PhaseA_Value","Vabc_loc_defect_FirstArmonic_PhaseA_Angle",
                          "Vabc_loc_defect_FirstArmonic_PhaseB_Value","Vabc_loc_defect_FirstArmonic_PhaseB_Angle",
                          "Vabc_loc_defect_FirstArmonic_PhaseC_Value","Vabc_loc_defect_FirstArmonic_PhaseC_Angle",
                    "Vabc_loc_defect_DeformingComponent_PhaseA","Vabc_loc_defect_DeformingComponent_PhaseB","Vabc_loc_defect_DeformingComponent_PhaseC",
                    "Vabc_loc_defect_CoefficientDistorsion_PhaseA","Vabc_loc_defect_CoefficientDistorsion_PhaseB","Vabc_loc_defect_CoefficientDistorsion_PhaseC",
                    "Vabc_loc_defect_CoefficientDistorsionContinue_PhaseA","Vabc_loc_defect_CoefficientDistorsionContinue_PhaseB","Vabc_loc_defect_CoefficientDistorsionContinue_PhaseC",
                    "Vabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseA","Vabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseB","Vabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Vabc_loc_defect_DirectComponent_Value","Vabc_loc_defect_DirectComponent_Angle",
                      "Vabc_loc_defect_InverseComponent_Value","Vabc_loc_defect_InverseComponent_Angle",
                      "Vabc_loc_defect_ZeroComponent_Value", "Vabc_loc_defect_ZeroComponent_Angle",
                      "Vabc_loc_defect_CoefficeintNegativeAsimetry","Vabc_loc_defect_CoefficeintZeroAsimetry"))  
    for i in range(len(result_Ic)):
        output_Ic += ComputeHarmonicAsymetrySingleFile(result_Ic[i][1],inputDir,N,frequency)[1:]
        output_Vc += ComputeHarmonicAsymetrySingleFile(result_Vc[i][1],inputDir,N,frequency)[1:]
        output_Ig += ComputeHarmonicAsymetrySingleFile(result_Ig[i][1],inputDir,N,frequency)[1:]
        output_Vg += ComputeHarmonicAsymetrySingleFile(result_Vg[i][1],inputDir,N,frequency)[1:]
        output_Id += ComputeHarmonicAsymetrySingleFile(result_Id[i][1],inputDir,N,frequency)[1:]
        output_Vd += ComputeHarmonicAsymetrySingleFile(result_Vd[i][1],inputDir,N,frequency)[1:]
    final_result = list()
    for i in range(len(output_Ic)):
        final_result.append(output_Ig[i] + output_Vg[i] + output_Id[i] + output_Vd[i] + output_Ic[i]+output_Vc[i])
    data = pd.DataFrame.from_records(final_result[1:], columns=final_result[0])
    isExist = os.path.exists(outputDir)
    if not isExist:
        os.makedirs(outputDir)
    outputDir = os.path.join(os.path.join(os.getcwd(), outputDir), outputFile)

    data.to_csv(outputDir)
    return data

In [ ]:
def SaveWaveletSingleFile(filename, scales=np.arange(1, 51), wavelets="gaus1", inputDir="ReadFiles", outputDir="OutputWavelet"):
    data = ReadSingleData(readSingleFile(fileName, inputDir))
    minTime = min(data[0])
    maxTime = max(data[0])
    minScales=min(scales)
    maxScales=max(scales)
    isExist = os.path.exists(outputDir)
    if not isExist:
        os.makedirs(outputDir)
    outputFile1 = os.path.join(os.path.join(os.getcwd(),outputDir),filename.replace('.txt','_Export_Data1.csv'))
    outputImage1 = os.path.join(os.path.join(os.getcwd(),outputDir),filename.replace('.txt','_Export_Wavelet1.png'))
    outputFile2 = os.path.join(os.path.join(os.getcwd(),outputDir),filename.replace('.txt','_Export_Data2.csv'))
    outputImage2 = os.path.join(os.path.join(os.getcwd(),outputDir),filename.replace('.txt','_Export_Wavelet2.png'))
    outputFile3 = os.path.join(os.path.join(os.getcwd(),outputDir),filename.replace('.txt','_Export_Data3.csv'))
    outputImage3 = os.path.join(os.path.join(os.getcwd(),outputDir),filename.replace('.txt','_Export_Wavelet3.png'))    
    coef1, freqs1 = pywt.cwt(data[1], scales, wavelets)
    coef2, freqs2 = pywt.cwt(data[2], scales, wavelets)
    coef3, freqs3 = pywt.cwt(data[3], scales, wavelets)
    plt.figure(figsize=(15, 10))
    plt.imshow(abs(coef1), extent=[minTime, maxTime, maxScales, minScales], interpolation='bilinear', cmap='bone',
               aspect='auto', vmax=abs(coef1).max(), vmin=abs(coef1).min())
    plt.gca().invert_yaxis()
    plt.savefig(outputImage1)
    plt.close()
    plt.figure(figsize=(15, 10))
    plt.imshow(abs(coef2), extent=[minTime, maxTime, maxScales, minScales], interpolation='bilinear', cmap='bone',
               aspect='auto', vmax=abs(coef2).max(), vmin=abs(coef2).min())
    plt.gca().invert_yaxis()
    plt.savefig(outputImage2)
    plt.close()
    plt.figure(figsize=(15, 10))
    plt.imshow(abs(coef3), extent=[minTime, maxTime, maxScales, minScales], interpolation='bilinear', cmap='bone',
               aspect='auto', vmax=abs(coef2).max(), vmin=abs(coef3).min())
    plt.gca().invert_yaxis()
    plt.savefig(outputImage3)
    plt.close()
    with open(outputFile1, mode='w') as outputFile:
        output_writer = csv.writer(outputFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        output_writer.writerow(['frequency'])
        output_writer.writerow(freqs1)
        output_writer.writerow(['coefficients'])
        for item in coef1:
            output_writer.writerow(item)
    with open(outputFile2, mode='w') as outputFile:
        output_writer = csv.writer(outputFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        output_writer.writerow(['frequency'])
        output_writer.writerow(freqs2)
        output_writer.writerow(['coefficients'])
        for item in coef2:
            output_writer.writerow(item)
    with open(outputFile3, mode='w') as outputFile:
        output_writer = csv.writer(outputFile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        output_writer.writerow(['frequency'])
        output_writer.writerow(freqs1)
        output_writer.writerow(['coefficients'])
        for item in coef3:
            output_writer.writerow(item)

In [ ]:
def SaveWaveletMultipleFile(inputDir="ReadFiles", outputDir="OutputFile", scales=np.arange(1, 51), wavelets="gaus1"):
    result=list()
    for folder,sub_folder,files in os.walk(os.path.join(os.getcwd(),inputDir)):
        for nume_fisier in files:
            key1,value=nume_fisier.split('_',1)
            result.append([int(key1),nume_fisier])
    result.sort()
    for nume in result:
        try:
            SaveWaveletSingleFile(nume[1],scales,wavelets,inputDir,outputDir)
        except:
            print(nume[1])
        

In [ ]:
#ExportAllFilesIntoOneCSV(inputDir="caz 01",outputDir="testare Model")
#ExportAllFilesIntoOneCSV(inputDir="caz 02",outputDir="testare Model")
ExportAllFilesIntoOneCSV_V3(inputDir="caz 03",outputDir="testare Model")
ExportAllFilesIntoOneCSV_V3(inputDir="caz 04",outputDir="testare Model")
ExportAllFilesIntoOneCSV_V3(inputDir="caz 05",outputDir="testare Model")
ExportAllFilesIntoOneCSV_V3(inputDir="caz 06",outputDir="testare Model")
ExportAllFilesIntoOneCSV_V3(inputDir="caz 07",outputDir="testare Model")
ExportAllFilesIntoOneCSV_V3(inputDir="caz 08",outputDir="testare Model")
ExportAllFilesIntoOneCSV_V3(inputDir="caz 09",outputDir="testare Model")
ExportAllFilesIntoOneCSV_V3(inputDir="caz 10",outputDir="testare Model")
ExportAllFilesIntoOneCSV_V3(inputDir="caz 11",outputDir="testare Model")
ExportAllFilesIntoOneCSV_V3(inputDir="caz 12",outputDir="testare Model")







In [ ]:
"""   """
#ExportAllFilesIntoOneCSV(inputDir="1 Scurtcircuit_monofazat_A_ground",outputDir="tema 5_400kV")
#ExportAllFilesIntoOneCSV(inputDir="2 Scurtcircuit_monofazat_B_ground",outputDir="tema 3_20kV_rezistiv_capacitiv")
#ExportAllFilesIntoOneCSV(inputDir="3 Scurtcircuit_monofazat_C_ground",outputDir="tema 3_20kV_rezistiv_capacitiv")
#ExportAllFilesIntoOneCSV(inputDir="4 Scurtcircuit_bifazat_AB_ground",outputDir="tema 5_400kV")
#ExportAllFilesIntoOneCSV(inputDir="5 Scurtcircuit_bifazat_BC_ground",outputDir="tema 3_20kV_rezistiv_capacitiv")
#ExportAllFilesIntoOneCSV(inputDir="6 Scurtcircuit_bifazat_AC_ground",outputDir="tema 3_20kV_rezistiv_capacitiv")
#ExportAllFilesIntoOneCSV(inputDir="7 Scurtcircuit_trifazat_ABC_ground",outputDir="tema 5_400kV")


In [ ]:

#SaveWaveletMultipleFile(inputDir="1 Scurtcircuit_monofazat_A_ground",outputDir="1 Scurtcircuit_monofazat_A_ground_ExportWavelet")
#SaveWaveletMultipleFile(inputDir="2 Scurtcircuit_monofazat_B_ground",outputDir="2 Scurtcircuit_monofazat_B_ground_ExportWavelet")
#SaveWaveletMultipleFile(inputDir="3 Scurtcircuit_monofazat_C_ground",outputDir="3 Scurtcircuit_monofazat_C_ground_ExportWavelet")
#SaveWaveletMultipleFile(inputDir="4 Scurtcircuit_bifazat_AB_ground",outputDir="4 Scurtcircuit_bifazat_AB_ground_ExportWavelet")
#SaveWaveletMultipleFile(inputDir="5 Scurtcircuit_bifazat_BC_ground",outputDir="5 Scurtcircuit_bifazat_BC_ground_ExportWavelet")
#SaveWaveletMultipleFile(inputDir="6 Scurtcircuit_bifazat_AC_ground",outputDir="6 Scurtcircuit_bifazat_AC_ground_ExportWavelet")
#SaveWaveletMultipleFile(inputDir="7 Scurtcircuit_trifazat_ABC_ground",outputDir="7 Scurtcircuit_trifazat_ABC_ground_ExportWavelet")


In [ ]:
def ExportToVSC_AllFiles(inputDir="ReadFiles", outputDir="OutputFile", N=1, frequency=50):
    outputFile = inputDir + '_ExportToCSV.csv'
    result=list()
    for folder,sub_folder,files in os.walk(os.path.join(os.getcwd(),inputDir)):
        result=list()
        for nume_fisier in files:
            key1,value=nume_fisier.split('_',1)
            result.append([int(key1),nume_fisier])
    result.sort()
    output=list()
    output.append(("fileName", "cycle","ContinueComponent_PhaseA","ContinueComponent_PhaseB","ContinueComponent_PhaseC",
                    "EffectiveComponent_PhaseA","EffectiveComponent_PhaseB", "EffectiveComponent_PhaseC",
                    "FirstArmonic_PhaseA_Value","FirstArmonic_PhaseA_Angle",
                          "FirstArmonic_PhaseB_Value","irstArmonic_PhaseB_Angle",
                          "FirstArmonic_PhaseC_Value","FirstArmonic_PhaseC_Angle",
                    "DeformingComponent_PhaseA","DeformingComponent_PhaseB","DeformingComponent_PhaseC",
                    "CoefficientDistorsion_PhaseA","CoefficientDistorsion_PhaseB","CoefficientDistorsion_PhaseC",
                    "CoefficientDistorsionContinue_PhaseA","CoefficientDistorsionContinue_PhaseB","CoefficientDistorsionContinue_PhaseC",
                    "CoefficientDistorsionFirstArmonic_PhaseA","CoefficientDistorsionFirstArmonic_PhaseB","CoefficientDistorsionFirstArmonic_PhaseC",
                    "DirectComponent_Value","DirectComponent_Angle","InverseComponent_Value",
                   "InverseComponent_Angle","ZeroComponent_Value","ZeroComponent_Angle","CoefficeintNegativeAsimetry","CoefficeintZeroAsimetry"))    
    for i in range(len(result)):
        output += ComputeHarmonicAsymetrySingleFile(result[i][1],inputDir,N,frequency)[1:]
    data = pd.DataFrame.from_records(output[1:], columns=output[0])
    data.insert(1, "FaultLocation_FromGenerator", data["fileName"].map(lambda x: float(x.split('_')[1])))
    data.insert(2, "FaultLocation_ToConsumer", data["fileName"].map(lambda x: float(x.split('_')[2])))
    isExist = os.path.exists(outputDir)
    if not isExist:
        os.makedirs(outputDir)
    outputDir = os.path.join(os.path.join(os.getcwd(), outputDir), outputFile)

    data.to_csv(outputDir)
    return data

In [ ]:
#ExportToVSC_AllFiles(inputDir="1 Scurtcircuit_monofazat_A_ground_I",outputDir="tema 1B_20kV_sarcina_rezistiva_evolutie tensiune")
#ExportToVSC_AllFiles(inputDir="6 Scurtcircuit_bifazat_AC_ground_I",outputDir="tema 1B_20kV_sarcina_rezistiva_evolutie tensiune")
#ExportToVSC_AllFiles(inputDir="7 Scurtcircuit_trifazat_ABC_ground_I",outputDir="tema 1B_20kV_sarcina_rezistiva_evolutie tensiune")

#ExportToVSC_AllFiles(inputDir="1 Scurtcircuit_monofazat_A_ground_U",outputDir="tema 1B_20kV_sarcina_rezistiva_evolutie tensiune")
#ExportToVSC_AllFiles(inputDir="6 Scurtcircuit_bifazat_AC_ground_U",outputDir="tema 1B_20kV_sarcina_rezistiva_evolutie tensiune")
#ExportToVSC_AllFiles(inputDir="7 Scurtcircuit_trifazat_ABC_ground_U",outputDir="tema 1B_20kV_sarcina_rezistiva_evolutie tensiune")

In [ ]:
def ExportAllFilesIntoOneCSV_V2(inputDir="ReadFiles", outputDir="OutputFile", N=1, frequency=50):
    outputFile = inputDir + '_ExportToCSV.csv'
    result_Ic=list()
    result_Vc=list()
    result_Ig=list()
    result_Vg=list()
    result_Id=list()
    result_Vd=list()
    for folder,sub_folder,files in os.walk(os.path.join(os.getcwd(),inputDir)):

        for nume_fisier in files:
            key1,value=nume_fisier.split('_',1)
            if (('consumator'.upper() in nume_fisier.upper()) and '_I'.upper() in nume_fisier.upper()):
                result_Ic.append([int(key1),nume_fisier])
            if (('consumator'.upper() in nume_fisier.upper()) and '_V'.upper() in nume_fisier.upper()):
                result_Vc.append([int(key1),nume_fisier])
            if (('generator'.upper() in nume_fisier.upper()) and '_I'.upper() in nume_fisier.upper()):
                result_Ig.append([int(key1),nume_fisier])
            if (('generator'.upper() in nume_fisier.upper()) and '_V'.upper() in nume_fisier.upper()):
                result_Vg.append([int(key1),nume_fisier])
            if (('defect'.upper() in nume_fisier.upper()) and '_I'.upper() in nume_fisier.upper()):
                result_Id.append([int(key1),nume_fisier])
            if (('defect'.upper() in nume_fisier.upper()) and '_V'.upper() in nume_fisier.upper()):
                result_Vd.append([int(key1),nume_fisier])
    result_Ic.sort()
    result_Vc.sort()
    result_Ig.sort()
    result_Vg.sort()
    
    result_Id.sort()
    result_Vd.sort()
    output_Ic=list()
    output_Vc=list()
    output_Ig=list()
    output_Vg=list()
    output_Id=list()
    output_Vd=list()
    output_Ic.append(("Iabc_consumator_fileName", "cycle","Iabc_consumator_ContinueComponent_PhaseA","Iabc_consumator_ContinueComponent_PhaseB","Iabc_consumator_ContinueComponent_PhaseC",
                    "Iabc_consumator_EffectiveComponent_PhaseA","Iabc_consumator_EffectiveComponent_PhaseB", "Iabc_consumator_EffectiveComponent_PhaseC",
                    "Iabc_consumator_FirstArmonic_PhaseA_Value","Iabc_consumator_FirstArmonic_PhaseA_Angle",
                          "Iabc_consumator_FirstArmonic_PhaseB_Value","Iabc_consumator_FirstArmonic_PhaseB_Angle",
                          "Iabc_consumator_FirstArmonic_PhaseC_Value","Iabc_consumator_FirstArmonic_PhaseC_Angle",
                    "Iabc_consumator_DeformingComponent_PhaseA","Iabc_consumator_DeformingComponent_PhaseB","Iabc_consumator_DeformingComponent_PhaseC",
                    "Iabc_consumator_CoefficientDistorsion_PhaseA","Iabc_consumator_CoefficientDistorsion_PhaseB","Iabc_consumator_CoefficientDistorsion_PhaseC",
                    "Iabc_consumator_CoefficientDistorsionContinue_PhaseA","Iabc_consumator_CoefficientDistorsionContinue_PhaseB","Iabc_consumator_CoefficientDistorsionContinue_PhaseC",
                    "Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA","Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB","Iabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Iabc_consumator_DirectComponent_Value","Iabc_consumator_DirectComponent_Angle",
                      "Iabc_consumator_InverseComponent_Value","Iabc_consumator_InverseComponent_Angle",
                      "Iabc_consumator_ZeroComponent_Value","Iabc_consumator_ZeroComponent_Angle",
                      "Iabc_consumator_CoefficeintNegativeAsimetry","Iabc_consumator_CoefficeintZeroAsimetry"))
    output_Vc.append(("Vabc_consumator_fileName", "cycle","Vabc_consumator_ContinueComponent_PhaseA","Vabc_consumator_ContinueComponent_PhaseB","Vabc_consumator_ContinueComponent_PhaseC",
                    "Vabc_consumator_EffectiveComponent_PhaseA","Vabc_consumator_EffectiveComponent_PhaseB", "Vabc_consumator_EffectiveComponent_PhaseC",
                    "Vabc_consumator_FirstArmonic_PhaseA_Value","Vabc_consumator_FirstArmonic_PhaseA_Angle",
                          "Vabc_consumator_FirstArmonic_PhaseB_Value","Vabc_consumator_FirstArmonic_PhaseB_Angle",
                          "Vabc_consumator_FirstArmonic_PhaseC_Value","Vabc_consumator_FirstArmonic_PhaseC_Angle",
                    "Vabc_consumator_DeformingComponent_PhaseA","Vabc_consumator_DeformingComponent_PhaseB","Vabc_consumator_DeformingComponent_PhaseC",
                    "Vabc_consumator_CoefficientDistorsion_PhaseA","Vabc_consumator_CoefficientDistorsion_PhaseB","Vabc_consumator_CoefficientDistorsion_PhaseC",
                    "Vabc_consumator_CoefficientDistorsionContinue_PhaseA","Vabc_consumator_CoefficientDistorsionContinue_PhaseB","Vabc_consumator_CoefficientDistorsionContinue_PhaseC",
                    "Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseA","Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseB","Vabc_consumator_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Vabc_consumator_DirectComponent_Value","Vabc_consumator_DirectComponent_Angle",
                      "Vabc_consumator_InverseComponent_Value","Vabc_consumator_InverseComponent_Angle",
                      "Vabc_consumator_ZeroComponent_Value","Vabc_consumator_ZeroComponent_Angle",
                      "Vabc_consumator_CoefficeintNegativeAsimetry","Vabc_consumator_CoefficeintZeroAsimetry"))
    output_Ig.append(("Iabc_generator_fileName", "cycle","Iabc_generator_ContinueComponent_PhaseA","Iabc_generator_ContinueComponent_PhaseB","Iabc_generator_ContinueComponent_PhaseC",
                    "Iabc_generator_EffectiveComponent_PhaseA","Iabc_generator_EffectiveComponent_PhaseB", "Iabc_generator_EffectiveComponent_PhaseC",
                    "Iabc_generator_FirstArmonic_PhaseA_Value","Iabc_generator_FirstArmonic_PhaseA_Angle",
                          "Iabc_generator_FirstArmonic_PhaseB_Value","Iabc_generator_FirstArmonic_PhaseB_Angle",
                          "Iabc_generator_FirstArmonic_PhaseC_Value","Iabc_generator_FirstArmonic_PhaseC_Angle",
                    "Iabc_generator_DeformingComponent_PhaseA","Iabc_generator_DeformingComponent_PhaseB","Iabc_generator_DeformingComponent_PhaseC",
                    "Iabc_generator_CoefficientDistorsion_PhaseA","Iabc_generator_CoefficientDistorsion_PhaseB","Iabc_generator_CoefficientDistorsion_PhaseC",
                    "Iabc_generator_CoefficientDistorsionContinue_PhaseA","Iabc_generator_CoefficientDistorsionContinue_PhaseB","Iabc_generator_CoefficientDistorsionContinue_PhaseC",
                    "Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseA","Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseB","Iabc_generator_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Iabc_generator_DirectComponent_Value","Iabc_generator_DirectComponent_Angle",
                      "Iabc_generator_InverseComponent_Value","Iabc_generator_InverseComponent_Angle",
                      "Iabc_generator_ZeroComponent_Value","Iabc_generator_ZeroComponent_Angle",
                      "Iabc_generator_CoefficeintNegativeAsimetry","Iabc_generator_CoefficeintZeroAsimetry"))
    output_Vg.append(("Vabc_generator_fileName", "cycle","Vabc_generator_ContinueComponent_PhaseA","Vabc_generator_ContinueComponent_PhaseB","Vabc_generator_ContinueComponent_PhaseC",
                    "Vabc_generator_EffectiveComponent_PhaseA","Vabc_generator_EffectiveComponent_PhaseB", "Vabc_generator_EffectiveComponent_PhaseC",
                    "Vabc_generator_FirstArmonic_PhaseA_Value","Vabc_generator_FirstArmonic_PhaseA_Angle",
                          "Vabc_generator_FirstArmonic_PhaseB_Value","Vabc_generator_FirstArmonic_PhaseB_Angle",
                          "Vabc_generator_FirstArmonic_PhaseC_Value","Vabc_generator_FirstArmonic_PhaseC_Angle",
                    "Vabc_generator_DeformingComponent_PhaseA","Vabc_generator_DeformingComponent_PhaseB","Vabc_generator_DeformingComponent_PhaseC",
                    "Vabc_generator_CoefficientDistorsion_PhaseA","Vabc_generator_CoefficientDistorsion_PhaseB","Vabc_generator_CoefficientDistorsion_PhaseC",
                    "Vabc_generator_CoefficientDistorsionContinue_PhaseA","Vabc_generator_CoefficientDistorsionContinue_PhaseB","Vabc_generator_CoefficientDistorsionContinue_PhaseC",
                    "Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseA","Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseB","Vabc_generator_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Vabc_generator_DirectComponent_Value","Vabc_generator_DirectComponent_Angle",
                      "Vabc_generator_InverseComponent_Value","Vabc_generator_InverseComponent_Angle",
                      "Vabc_generator_ZeroComponent_Value","Vabc_generator_ZeroComponent_Angle",
                      "Vabc_generator_CoefficeintNegativeAsimetry","Vabc_generator_CoefficeintZeroAsimetry"))
    output_Id.append(("Iabc_loc_defect_fileName", "cycle","Iabc_loc_defect_ContinueComponent_PhaseA","Iabc_loc_defect_ContinueComponent_PhaseB","Iabc_loc_defect_ContinueComponent_PhaseC",
                    "Iabc_loc_defect_EffectiveComponent_PhaseA","Iabc_loc_defect_EffectiveComponent_PhaseB", "Iabc_loc_defect_EffectiveComponent_PhaseC",
                    "Iabc_loc_defect_FirstArmonic_PhaseA_Value","Iabc_loc_defect_FirstArmonic_PhaseA_Angle",
                          "Iabc_loc_defect_FirstArmonic_PhaseB_Value","Iabc_loc_defect_FirstArmonic_PhaseB_Angle",
                          "Iabc_loc_defect_FirstArmonic_PhaseC_Value","Iabc_loc_defect_FirstArmonic_PhaseC_Angle",
                    "Iabc_loc_defect_DeformingComponent_PhaseA","Iabc_loc_defect_DeformingComponent_PhaseB","Iabc_loc_defect_DeformingComponent_PhaseC",
                    "Iabc_loc_defect_CoefficientDistorsion_PhaseA","Iabc_loc_defect_CoefficientDistorsion_PhaseB","Iabc_loc_defect_CoefficientDistorsion_PhaseC",
                    "Iabc_loc_defect_CoefficientDistorsionContinue_PhaseA","Iabc_loc_defect_CoefficientDistorsionContinue_PhaseB","Iabc_loc_defect_CoefficientDistorsionContinue_PhaseC",
                    "Iabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseA","Iabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseB","Iabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Iabc_loc_defect_DirectComponent_Value","Iabc_loc_defect_DirectComponent_Angle",
                      "Iabc_loc_defect_InverseComponent_Value","Iabc_loc_defect_InverseComponent_Angle",
                      "Iabc_loc_defect_ZeroComponent_Value","Iabc_loc_defect_ZeroComponent_Angle",
                      "Iabc_loc_defect_CoefficeintNegativeAsimetry","Iabc_loc_defect_CoefficeintZeroAsimetry"))
    output_Vd.append(("Vabc_loc_defect_fileName", "cycle","Vabc_loc_defect_ContinueComponent_PhaseA","Vabc_loc_defect_ContinueComponent_PhaseB","Vabc_loc_defect_ContinueComponent_PhaseC",
                    "Vabc_loc_defect_EffectiveComponent_PhaseA","Vabc_loc_defect_EffectiveComponent_PhaseB", "Vabc_loc_defect_EffectiveComponent_PhaseC",
                    "Vabc_loc_defect_FirstArmonic_PhaseA_Value","Vabc_loc_defect_FirstArmonic_PhaseA_Angle",
                          "Vabc_loc_defect_FirstArmonic_PhaseB_Value","Vabc_loc_defect_FirstArmonic_PhaseB_Angle",
                          "Vabc_loc_defect_FirstArmonic_PhaseC_Value","Vabc_loc_defect_FirstArmonic_PhaseC_Angle",
                    "Vabc_loc_defect_DeformingComponent_PhaseA","Vabc_loc_defect_DeformingComponent_PhaseB","Vabc_loc_defect_DeformingComponent_PhaseC",
                    "Vabc_loc_defect_CoefficientDistorsion_PhaseA","Vabc_loc_defect_CoefficientDistorsion_PhaseB","Vabc_loc_defect_CoefficientDistorsion_PhaseC",
                    "Vabc_loc_defect_CoefficientDistorsionContinue_PhaseA","Vabc_loc_defect_CoefficientDistorsionContinue_PhaseB","Vabc_loc_defect_CoefficientDistorsionContinue_PhaseC",
                    "Vabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseA","Vabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseB","Vabc_loc_defect_CoefficientDistorsionFirstArmonic_PhaseC",
                    "Vabc_loc_defect_DirectComponent_Value","Vabc_loc_defect_DirectComponent_Angle",
                      "Vabc_loc_defect_InverseComponent_Value","Vabc_loc_defect_InverseComponent_Angle",
                      "Vabc_loc_defect_ZeroComponent_Value","Vabc_loc_defect_ZeroComponent_Angle",
                      "Vabc_loc_defect_CoefficeintNegativeAsimetry","Vabc_loc_defect_CoefficeintZeroAsimetry"))  
    for i in range(len(result_Ic)):
        output_Ic += ComputeHarmonicAsymetrySingleFile(result_Ic[i][1],inputDir,N,frequency)[1:]
        output_Vc += ComputeHarmonicAsymetrySingleFile(result_Vc[i][1],inputDir,N,frequency)[1:]
        output_Ig += ComputeHarmonicAsymetrySingleFile(result_Ig[i][1],inputDir,N,frequency)[1:]
        output_Vg += ComputeHarmonicAsymetrySingleFile(result_Vg[i][1],inputDir,N,frequency)[1:]
        output_Id += ComputeHarmonicAsymetrySingleFile(result_Id[i][1],inputDir,N,frequency)[1:]
        output_Vd += ComputeHarmonicAsymetrySingleFile(result_Vd[i][1],inputDir,N,frequency)[1:]
    final_result = list()
    for i in range(len(output_Ic)):
        final_result.append(output_Ig[i]+output_Vg[i] + output_Id[i]+output_Vd[i] + output_Ic[i]+output_Vc[i])
    data = pd.DataFrame.from_records(final_result[1:], columns=final_result[0])
    data.insert(1, "FaultLocation_FromGenerator", data["Iabc_consumator_fileName"].map(lambda x: float(x.split('_')[1])))
    data.insert(2, "FaultLocation_ToConsumer", data["Iabc_consumator_fileName"].map(lambda x: float(x.split('_')[2])))
    isExist = os.path.exists(outputDir)
    if not isExist:
        os.makedirs(outputDir)
    outputDir = os.path.join(os.path.join(os.getcwd(), outputDir), outputFile)

    data.to_csv(outputDir)
    return data

In [ ]:
#ExportAllFilesIntoOneCSV_V2(inputDir="bifazat AC 50_250", outputDir="tema 6 400 kv_linie lunga")
#ExportAllFilesIntoOneCSV_V2(inputDir="monofazat A_50_250", outputDir="tema 6 400 kv_linie lunga")
#ExportAllFilesIntoOneCSV_V2(inputDir="monofazat A_50_450", outputDir="tema 6 400 kv_linie lunga")
#ExportAllFilesIntoOneCSV_V2(inputDir="trifazat ABC 50_250", outputDir="tema 6 400 kv_linie lunga")

In [ ]:
def ExportToCSV_AllFiles_Fourier(inputDir="ReadFiles", outputDir="OutputFile", frequency=50):
    outputFile = inputDir + '_ExportToCSV.csv'
    result=list()
    for folder,sub_folder,files in os.walk(os.path.join(os.getcwd(),inputDir)):
        result=list()
        for nume_fisier in files:
            key1,value=nume_fisier.split('_',1)
            result.append([int(key1), float(nume_fisier[nume_fisier.find("_D")+2:-6]),nume_fisier])
    result.sort()
    output=list()
    output.append(("fileName", "cycle",
    "Armonic1_PhaseA_Value","Armonic2_PhaseA_Value","Armonic3_PhaseA_Value",
    "Armonic4_PhaseA_Value","Armonic5_PhaseA_Value","Armonic6_PhaseA_Value",
    "Armonic7_PhaseA_Value","Armonic8_PhaseA_Value","Armonic9_PhaseA_Value",
    "Armonic10_PhaseA_Value",
    "Armonic11_PhaseA_Value","Armonic12_PhaseA_Value","Armonic13_PhaseA_Value",
    "Armonic14_PhaseA_Value","Armonic15_PhaseA_Value","Armonic16_PhaseA_Value",
    "Armonic17_PhaseA_Value","Armonic18_PhaseA_Value","Armonic19_PhaseA_Value",
    "Armonic20_PhaseA_Value",
    "Armonic21_PhaseA_Value","Armonic22_PhaseA_Value","Armonic23_PhaseA_Value",
    "Armonic24_PhaseA_Value","Armonic25_PhaseA_Value","Armonic26_PhaseA_Value",
    "Armonic27_PhaseA_Value","Armonic28_PhaseA_Value","Armonic29_PhaseA_Value",
    "Armonic30_PhaseA_Value",
    "Armonic31_PhaseA_Value","Armonic32_PhaseA_Value","Armonic33_PhaseA_Value",
    "Armonic34_PhaseA_Value","Armonic35_PhaseA_Value","Armonic36_PhaseA_Value",
    "Armonic37_PhaseA_Value","Armonic38_PhaseA_Value","Armonic39_PhaseA_Value",
    "Armonic40_PhaseA_Value",
    "Armonic41_PhaseA_Value","Armonic42_PhaseA_Value","Armonic43_PhaseA_Value",
    "Armonic44_PhaseA_Value","Armonic45_PhaseA_Value","Armonic46_PhaseA_Value",
    "Armonic47_PhaseA_Value","Armonic48_PhaseA_Value","Armonic49_PhaseA_Value",
    "Armonic50_PhaseA_Value",
    "Armonic1_PhaseB_Value","Armonic2_PhaseB_Value","Armonic3_PhaseB_Value",
    "Armonic4_PhaseB_Value","Armonic5_PhaseB_Value","Armonic6_PhaseB_Value",
    "Armonic7_PhaseB_Value","Armonic8_PhaseB_Value","Armonic9_PhaseB_Value",
    "Armonic10_PhaseB_Value",
    "Armonic11_PhaseB_Value","Armonic12_PhaseB_Value","Armonic13_PhaseB_Value",
    "Armonic14_PhaseB_Value","Armonic15_PhaseB_Value","Armonic16_PhaseB_Value",
    "Armonic17_PhaseB_Value","Armonic18_PhaseB_Value","Armonic19_PhaseB_Value",
    "Armonic20_PhaseB_Value",
    "Armonic21_PhaseB_Value","Armonic22_PhaseB_Value","Armonic23_PhaseB_Value",
    "Armonic24_PhaseB_Value","Armonic25_PhaseB_Value","Armonic26_PhaseB_Value",
    "Armonic27_PhaseB_Value","Armonic28_PhaseB_Value","Armonic29_PhaseB_Value",
    "Armonic30_PhaseB_Value",
    "Armonic31_PhaseB_Value","Armonic32_PhaseB_Value","Armonic33_PhaseB_Value",
    "Armonic34_PhaseB_Value","Armonic35_PhaseB_Value","Armonic36_PhaseB_Value",
    "Armonic37_PhaseB_Value","Armonic38_PhaseB_Value","Armonic39_PhaseB_Value",
    "Armonic40_PhaseB_Value",
    "Armonic41_PhaseB_Value","Armonic42_PhaseB_Value","Armonic43_PhaseB_Value",
    "Armonic44_PhaseB_Value","Armonic45_PhaseB_Value","Armonic46_PhaseB_Value",
    "Armonic47_PhaseB_Value","Armonic48_PhaseB_Value","Armonic49_PhaseB_Value",
    "Armonic50_PhaseB_Value",
    "Armonic1_PhaseC_Value","Armonic2_PhaseC_Value","Armonic3_PhaseC_Value",
    "Armonic4_PhaseC_Value","Armonic5_PhaseC_Value","Armonic6_PhaseC_Value",
    "Armonic7_PhaseC_Value","Armonic8_PhaseC_Value","Armonic9_PhaseC_Value",
    "Armonic10_PhaseC_Value",
    "Armonic11_PhaseC_Value","Armonic12_PhaseC_Value","Armonic13_PhaseC_Value",
    "Armonic14_PhaseC_Value","Armonic15_PhaseC_Value","Armonic16_PhaseC_Value",
    "Armonic17_PhaseC_Value","Armonic18_PhaseC_Value","Armonic19_PhaseC_Value",
    "Armonic20_PhaseC_Value",
    "Armonic21_PhaseC_Value","Armonic22_PhaseC_Value","Armonic23_PhaseC_Value",
    "Armonic24_PhaseC_Value","Armonic25_PhaseC_Value","Armonic26_PhaseC_Value",
    "Armonic27_PhaseC_Value","Armonic28_PhaseC_Value","Armonic29_PhaseC_Value",
    "Armonic30_PhaseC_Value",
    "Armonic31_PhaseC_Value","Armonic32_PhaseC_Value","Armonic33_PhaseC_Value",
    "Armonic34_PhaseC_Value","Armonic35_PhaseC_Value","Armonic36_PhaseC_Value",
    "Armonic37_PhaseC_Value","Armonic38_PhaseC_Value","Armonic39_PhaseC_Value",
    "Armonic40_PhaseC_Value",
    "Armonic41_PhaseC_Value","Armonic42_PhaseC_Value","Armonic43_PhaseC_Value",
    "Armonic44_PhaseC_Value","Armonic45_PhaseC_Value","Armonic46_PhaseC_Value",
    "Armonic47_PhaseC_Value","Armonic48_PhaseC_Value","Armonic49_PhaseC_Value",
    "Armonic50_PhaseC_Value",
    "Armonic1_PhaseA_Angle","Armonic2_PhaseA_Angle","Armonic3_PhaseA_Angle",
    "Armonic4_PhaseA_Angle","Armonic5_PhaseA_Angle","Armonic6_PhaseA_Angle",
    "Armonic7_PhaseA_Angle","Armonic8_PhaseA_Angle","Armonic9_PhaseA_Angle",
    "Armonic10_PhaseA_Angle",
    "Armonic11_PhaseA_Angle","Armonic12_PhaseA_Angle","Armonic13_PhaseA_Angle",
    "Armonic14_PhaseA_Angle","Armonic15_PhaseA_Angle","Armonic16_PhaseA_Angle",
    "Armonic17_PhaseA_Angle","Armonic18_PhaseA_Angle","Armonic19_PhaseA_Angle",
    "Armonic20_PhaseA_Angle",
    "Armonic21_PhaseA_Angle","Armonic22_PhaseA_Angle","Armonic23_PhaseA_Angle",
    "Armonic24_PhaseA_Angle","Armonic25_PhaseA_Angle","Armonic26_PhaseA_Angle",
    "Armonic27_PhaseA_Angle","Armonic28_PhaseA_Angle","Armonic29_PhaseA_Angle",
    "Armonic30_PhaseA_Angle",
    "Armonic31_PhaseA_Angle","Armonic32_PhaseA_Angle","Armonic33_PhaseA_Angle",
    "Armonic34_PhaseA_Angle","Armonic35_PhaseA_Angle","Armonic36_PhaseA_Angle",
    "Armonic37_PhaseA_Angle","Armonic38_PhaseA_Angle","Armonic39_PhaseA_Angle",
    "Armonic40_PhaseA_Angle",
    "Armonic41_PhaseA_Angle","Armonic42_PhaseA_Angle","Armonic43_PhaseA_Angle",
    "Armonic44_PhaseA_Angle","Armonic45_PhaseA_Angle","Armonic46_PhaseA_Angle",
    "Armonic47_PhaseA_Angle","Armonic48_PhaseA_Angle","Armonic49_PhaseA_Angle",
    "Armonic50_PhaseA_Angle",
    "Armonic1_PhaseB_Angle","Armonic2_PhaseB_Angle","Armonic3_PhaseB_Angle",
    "Armonic4_PhaseB_Angle","Armonic5_PhaseB_Angle","Armonic6_PhaseB_Angle",
    "Armonic7_PhaseB_Angle","Armonic8_PhaseB_Angle","Armonic9_PhaseB_Angle",
    "Armonic10_PhaseB_Angle",
    "Armonic11_PhaseB_Angle","Armonic12_PhaseB_Angle","Armonic13_PhaseB_Angle",
    "Armonic14_PhaseB_Angle","Armonic15_PhaseB_Angle","Armonic16_PhaseB_Angle",
    "Armonic17_PhaseB_Angle","Armonic18_PhaseB_Angle","Armonic19_PhaseB_Angle",
    "Armonic20_PhaseB_Angle",
    "Armonic21_PhaseB_Angle","Armonic22_PhaseB_Angle","Armonic23_PhaseB_Angle",
    "Armonic24_PhaseB_Angle","Armonic25_PhaseB_Angle","Armonic26_PhaseB_Angle",
    "Armonic27_PhaseB_Angle","Armonic28_PhaseB_Angle","Armonic29_PhaseB_Angle",
    "Armonic30_PhaseB_Angle",
    "Armonic31_PhaseB_Angle","Armonic32_PhaseB_Angle","Armonic33_PhaseB_Angle",
    "Armonic34_PhaseB_Angle","Armonic35_PhaseB_Angle","Armonic36_PhaseB_Angle",
    "Armonic37_PhaseB_Angle","Armonic38_PhaseB_Angle","Armonic39_PhaseB_Angle",
    "Armonic40_PhaseB_Angle",
    "Armonic41_PhaseB_Angle","Armonic42_PhaseB_Angle","Armonic43_PhaseB_Angle",
    "Armonic44_PhaseB_Angle","Armonic45_PhaseB_Angle","Armonic46_PhaseB_Angle",
    "Armonic47_PhaseB_Angle","Armonic48_PhaseB_Angle","Armonic49_PhaseB_Angle",
    "Armonic50_PhaseB_Angle",
    "Armonic1_PhaseC_Angle","Armonic2_PhaseC_Angle","Armonic3_PhaseC_Angle",
    "Armonic4_PhaseC_Angle","Armonic5_PhaseC_Angle","Armonic6_PhaseC_Angle",
    "Armonic7_PhaseC_Angle","Armonic8_PhaseC_Angle","Armonic9_PhaseC_Angle",
    "Armonic10_PhaseC_Angle",
    "Armonic11_PhaseC_Angle","Armonic12_PhaseC_Angle","Armonic13_PhaseC_Angle",
    "Armonic14_PhaseC_Angle","Armonic15_PhaseC_Angle","Armonic16_PhaseC_Angle",
    "Armonic17_PhaseC_Angle","Armonic18_PhaseC_Angle","Armonic19_PhaseC_Angle",
    "Armonic20_PhaseC_Angle",
    "Armonic21_PhaseC_Angle","Armonic22_PhaseC_Angle","Armonic23_PhaseC_Angle",
    "Armonic24_PhaseC_Angle","Armonic25_PhaseC_Angle","Armonic26_PhaseC_Angle",
    "Armonic27_PhaseC_Angle","Armonic28_PhaseC_Angle","Armonic29_PhaseC_Angle",
    "Armonic30_PhaseC_Angle",
    "Armonic31_PhaseC_Angle","Armonic32_PhaseC_Angle","Armonic33_PhaseC_Angle",
    "Armonic34_PhaseC_Angle","Armonic35_PhaseC_Angle","Armonic36_PhaseC_Angle",
    "Armonic37_PhaseC_Angle","Armonic38_PhaseC_Angle","Armonic39_PhaseC_Angle",
    "Armonic40_PhaseC_Angle",
    "Armonic41_PhaseC_Angle","Armonic42_PhaseC_Angle","Armonic43_PhaseC_Angle",
    "Armonic44_PhaseC_Angle","Armonic45_PhaseC_Angle","Armonic46_PhaseC_Angle",
    "Armonic47_PhaseC_Angle","Armonic48_PhaseC_Angle","Armonic49_PhaseC_Angle",
    "Armonic50_PhaseC_Angle"))
    for i in range(len(result)):
        input_data=ReadSingleData(readSingleFile(result[i][2],inputDir))
        A1 = ExtractSingleData(input_data,1,frequency)
        A2 = ExtractSingleData(input_data,2,frequency)
        A3 = ExtractSingleData(input_data,3,frequency)
        A4 = ExtractSingleData(input_data,4,frequency)
        A5 = ExtractSingleData(input_data,5,frequency)
        A6 = ExtractSingleData(input_data,6,frequency)
        A7 = ExtractSingleData(input_data,7,frequency)
        A8 = ExtractSingleData(input_data,8,frequency)
        A9 = ExtractSingleData(input_data,9,frequency)
        A10 = ExtractSingleData(input_data,10,frequency)
        A11 = ExtractSingleData(input_data,11,frequency)
        A12 = ExtractSingleData(input_data,12,frequency)
        A13 = ExtractSingleData(input_data,13,frequency)
        A14 = ExtractSingleData(input_data,14,frequency)
        A15 = ExtractSingleData(input_data,15,frequency)
        A16 = ExtractSingleData(input_data,16,frequency)
        A17 = ExtractSingleData(input_data,17,frequency)
        A18 = ExtractSingleData(input_data,18,frequency)
        A19 = ExtractSingleData(input_data,19,frequency)
        A20 = ExtractSingleData(input_data,20,frequency)
        A21 = ExtractSingleData(input_data,21,frequency)
        A22 = ExtractSingleData(input_data,22,frequency)
        A23 = ExtractSingleData(input_data,23,frequency)
        A24 = ExtractSingleData(input_data,24,frequency)
        A25 = ExtractSingleData(input_data,25,frequency)
        A26 = ExtractSingleData(input_data,26,frequency)
        A27 = ExtractSingleData(input_data,27,frequency)
        A28 = ExtractSingleData(input_data,28,frequency)
        A29 = ExtractSingleData(input_data,29,frequency)
        A30 = ExtractSingleData(input_data,30,frequency)
        A31 = ExtractSingleData(input_data,31,frequency)
        A32 = ExtractSingleData(input_data,32,frequency)
        A33 = ExtractSingleData(input_data,33,frequency)
        A34 = ExtractSingleData(input_data,34,frequency)
        A35 = ExtractSingleData(input_data,35,frequency)
        A36 = ExtractSingleData(input_data,36,frequency)
        A37 = ExtractSingleData(input_data,37,frequency)
        A38 = ExtractSingleData(input_data,38,frequency)
        A39 = ExtractSingleData(input_data,39,frequency)
        A40 = ExtractSingleData(input_data,40,frequency)
        A41 = ExtractSingleData(input_data,41,frequency)
        A42 = ExtractSingleData(input_data,42,frequency)
        A43 = ExtractSingleData(input_data,43,frequency)
        A44 = ExtractSingleData(input_data,44,frequency)
        A45 = ExtractSingleData(input_data,45,frequency)
        A46 = ExtractSingleData(input_data,46,frequency)
        A47 = ExtractSingleData(input_data,47,frequency)
        A48 = ExtractSingleData(input_data,48,frequency)
        A49 = ExtractSingleData(input_data,49,frequency)
        A50 = ExtractSingleData(input_data,50,frequency)
        itemI = list()
        for j in range(len(A1)):
            itemI.append((result[i][2],A1[j][0],
                        A1[j][1],A2[j][1],A3[j][1],A4[j][1],A5[j][1],A6[j][1],A7[j][1],A8[j][1],A9[j][1]
             ,A10[j][1],A11[j][1],A12[j][1],A13[j][1],A14[j][1],A15[j][1],A16[j][1],A17[j][1],A18[j][1],A19[j][1]
             ,A20[j][1],A21[j][1],A22[j][1],A23[j][1],A24[j][1],A25[j][1],A26[j][1],A27[j][1],A28[j][1],A29[j][1]                        
             ,A30[j][1],A31[j][1],A32[j][1],A33[j][1],A34[j][1],A35[j][1],A36[j][1],A37[j][1],A38[j][1],A39[j][1]
             ,A40[j][1],A41[j][1],A42[j][1],A43[j][1],A44[j][1],A45[j][1],A46[j][1],A47[j][1],A48[j][1],A49[j][1],A50[j][1] 
                        ,A1[j][3],A2[j][3],A3[j][3],A4[j][3],A5[j][3],A6[j][3],A7[j][3],A8[j][3],A9[j][3]
             ,A10[j][3],A11[j][3],A12[j][3],A13[j][3],A14[j][3],A15[j][3],A16[j][3],A17[j][3],A18[j][3],A19[j][3]
             ,A20[j][3],A21[j][3],A22[j][3],A23[j][3],A24[j][3],A25[j][3],A26[j][3],A27[j][3],A28[j][3],A29[j][3]                        
             ,A30[j][3],A31[j][3],A32[j][3],A33[j][3],A34[j][3],A35[j][3],A36[j][3],A37[j][3],A38[j][3],A39[j][3]
             ,A40[j][3],A41[j][3],A42[j][3],A43[j][3],A44[j][3],A45[j][3],A46[j][3],A47[j][3],A48[j][3],A49[j][3],A50[j][3]             
                        ,A1[j][5],A2[j][5],A3[j][5],A4[j][5],A5[j][5],A6[j][5],A7[j][5],A8[j][5],A9[j][5]
             ,A10[j][5],A11[j][5],A12[j][5],A13[j][5],A14[j][5],A15[j][5],A16[j][5],A17[j][5],A18[j][5],A19[j][5]
             ,A20[j][5],A21[j][5],A22[j][5],A23[j][5],A24[j][5],A25[j][5],A26[j][5],A27[j][5],A28[j][5],A29[j][5]                        
             ,A30[j][5],A31[j][5],A32[j][5],A33[j][5],A34[j][5],A35[j][5],A36[j][5],A37[j][5],A38[j][5],A39[j][5]
             ,A40[j][5],A41[j][5],A42[j][5],A43[j][5],A44[j][5],A45[j][5],A46[j][5],A47[j][5],A48[j][5],A49[j][5],A50[j][5]

                        ,A1[j][2],A2[j][2],A3[j][2],A4[j][2],A5[j][2],A6[j][2],A7[j][2],A8[j][2],A9[j][2]
             ,A10[j][2],A11[j][2],A12[j][2],A13[j][2],A14[j][2],A15[j][2],A16[j][2],A17[j][2],A18[j][2],A19[j][2]
             ,A20[j][2],A21[j][2],A22[j][2],A23[j][2],A24[j][2],A25[j][2],A26[j][2],A27[j][2],A28[j][2],A29[j][2]                        
             ,A30[j][2],A31[j][2],A32[j][2],A33[j][2],A34[j][2],A35[j][2],A36[j][2],A37[j][2],A38[j][2],A39[j][2]
             ,A40[j][2],A41[j][2],A42[j][2],A43[j][2],A44[j][2],A45[j][2],A46[j][2],A47[j][2],A48[j][2],A49[j][2],A50[j][2]  
                        ,A1[j][4],A2[j][4],A3[j][4],A4[j][4],A5[j][4],A6[j][4],A7[j][4],A8[j][4],A9[j][4]
             ,A10[j][4],A11[j][4],A12[j][4],A13[j][4],A14[j][4],A15[j][4],A16[j][4],A17[j][4],A18[j][4],A19[j][4]
             ,A20[j][4],A21[j][4],A22[j][4],A23[j][4],A24[j][4],A25[j][4],A26[j][4],A27[j][4],A28[j][4],A29[j][4]                        
             ,A30[j][4],A31[j][4],A32[j][4],A33[j][4],A34[j][4],A35[j][4],A36[j][4],A37[j][4],A38[j][4],A39[j][4]
             ,A40[j][4],A41[j][4],A42[j][4],A43[j][4],A44[j][4],A45[j][4],A46[j][4],A47[j][4],A48[j][4],A49[j][4],A50[j][4] 
                        ,A1[j][6],A2[j][6],A3[j][6],A4[j][6],A5[j][6],A6[j][6],A7[j][6],A8[j][6],A9[j][6]
             ,A10[j][6],A11[j][6],A12[j][6],A13[j][6],A14[j][6],A15[j][6],A16[j][6],A17[j][6],A18[j][6],A19[j][6]
             ,A20[j][6],A21[j][6],A22[j][6],A23[j][6],A24[j][6],A25[j][6],A26[j][6],A27[j][6],A28[j][6],A29[j][6]                        
             ,A30[j][6],A31[j][6],A32[j][6],A33[j][6],A34[j][6],A35[j][6],A36[j][6],A37[j][6],A38[j][6],A39[j][6]
             ,A40[j][6],A41[j][6],A42[j][6],A43[j][6],A44[j][6],A45[j][6],A46[j][6],A47[j][6],A48[j][6],A49[j][6],A50[j][6] 
                        ))
        output += itemI
    data = pd.DataFrame.from_records(output[1:], columns=output[0])
    data.insert(1, "FaultLocation_FromGenerator", data["fileName"].map(lambda x: float(x.split('_')[1])))
    data.insert(2, "FaultLocation_ToConsumer", data["fileName"].map(lambda x: float(x.split('_')[2])))
    isExist = os.path.exists(outputDir)
    if not isExist:
        os.makedirs(outputDir)
    outputDir = os.path.join(os.path.join(os.getcwd(), outputDir), outputFile)

    data.to_csv(outputDir)
    return data

In [ ]:
ExportToCSV_AllFiles_Fourier(inputDir="1 Scurtcircuit_monofazat_A_ground_I",outputDir="tema 1C_20kV_sarcina_rezistiva_evolutie tensiune")
ExportToCSV_AllFiles_Fourier(inputDir="1 Scurtcircuit_monofazat_A_ground_U",outputDir="tema 1C_20kV_sarcina_rezistiva_evolutie tensiune")


In [ ]:
ExportToCSV_AllFiles_Fourier(inputDir="6 Scurtcircuit_bifazat_AC_ground_I",outputDir="tema 1C_20kV_sarcina_rezistiva_evolutie tensiune")
ExportToCSV_AllFiles_Fourier(inputDir="6 Scurtcircuit_bifazat_AC_ground_U",outputDir="tema 1C_20kV_sarcina_rezistiva_evolutie tensiune")


In [ ]:
ExportToCSV_AllFiles_Fourier(inputDir="7 Scurtcircuit_trifazat_ABC_ground_I",outputDir="tema 1C_20kV_sarcina_rezistiva_evolutie tensiune")
ExportToCSV_AllFiles_Fourier(inputDir="7 Scurtcircuit_trifazat_ABC_ground_U",outputDir="tema 1C_20kV_sarcina_rezistiva_evolutie tensiune")
